# This notebook shows how to use Orchestrator APIs for user experiments

## Configure the Environment

## Setup the Experiment

#### Import FABRIC API

In [1]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,1630021f-0a0c-4792-a241-997f410d36e1
Bastion Username,minawm_0041350787
Bastion Private Key File,/home/fabric/work/fabric_config/Mina_Bastion_Key_8
Bastion Host,bastion-2.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,1630021f-0a0c-4792-a241-997f410d36e1
Bastion Username,minawm_0041350787
Bastion Private Key File,/home/fabric/work/fabric_config/Mina_Bastion_Key_8
Bastion Host,bastion-2.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Create Slice
In Release 1.0, user is expected to assign the IP addresses manually. Please use the example comands indicated below:

In [10]:
import time
from IPython.display import clear_output

attempt_count = 0
restart = True
while restart :
    clear_output()
    try:
        available_resources = fablib.get_available_resources()
        print(f"Available Resources: {available_resources}")
        # available_resources.draw()
        restart = False
    except Exception as e:
        print("Attempt " + str(attempt_count) + " failed")
        attempt_count += 1
        print(f"Error: {e}")
        time.sleep(10)

Available Resources: Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
LOSA         6  192/192  1536/1536  60600/60600    381/381                 2/2                             2/2                            10/10               0/0               0/0
MASS         6  172/192  1488/1536  60470/60600    379/381                 2/2                             2/2                            9/10                2/2               3/3
WASH         6  94/192   1312/1536  58780/60600    364/381                 0/2                             2/2                            9/10                2/2               3/3
FIU         10  226/320  2264/2560  114290/116400  615/63

## Configure Slice Parameters



In [2]:
slice_name = 'MultiCloud2'
site = 'FIU' #'TACC' #'MAX' #'MASS' #'TACC' #'MASS' #'TACC' #'MAX' #'MASS' #'TACC'
    # "# MAX, TACC, MASS support v4 \n", #can use dedicated NICs with MAX.
    #IPv4 'FIU'
    

username='ubuntu'
image = 'default_ubuntu_20'
image_type = 'qcow2'

cloud_cores = 4
cloud_ram = 16
cloud_disk = 10

other_cores = 4 # 2
other_ram = 16 # 4
other_disk = 10

# bridge1_name ='bridge_c1_scheduler'
# bridge2_name ='bridge_c2_scheduler'
# bridge3_name ='bridge_c1_db1'
# bridge4_name ='bridge_c1_db2'
# bridge5_name ='bridge_c2_db1'
# bridge6_name ='bridge_c2_db2'

# nic1_name = 'c1_db_nic'
# nic2_name = 'c1_scheduler_nic'
# nic3_name = 'c2_db_nic'
# nic4_name = 'c2_scheduler_nic'
# nic5_name = 'db1_nic'
# nic6_name = 'db2_nic'
# nic7_name = 'scheduler_nic'
# nic8_name = 'scheduler_nic2'

In [16]:
try:
    #Create Slice
    slice1 = fablib.new_slice(name=slice_name)

    
    #nodes
    scheduler = slice1.add_node(name='scheduler_node', site=site)
    scheduler.set_capacities(cores=other_cores, ram=other_ram, disk=other_disk)
    scheduler.set_image(image)
    
    cloud1 = slice1.add_node(name='cloud1_node', site=site)
    cloud1.set_capacities(cores=cloud_cores, ram=cloud_ram, disk=cloud_disk)
    cloud1.set_image(image)
        
    cloud2 = slice1.add_node(name='cloud2_node', site=site)
    cloud2.set_capacities(cores=cloud_cores, ram=cloud_ram, disk=cloud_disk)
    cloud2.set_image(image)
        
    database1 = slice1.add_node(name='database1_node', site=site)
    database1.set_capacities(cores=other_cores, ram=other_ram, disk=other_disk)
    database1.set_image(image)
    
    database2 = slice1.add_node(name='database2_node', site=site)
    database2.set_capacities(cores=other_cores, ram=other_ram, disk=other_disk)
    database2.set_image(image)
    
    
    #NICs

#     c1_db_nic = cloud1.add_component(model='NIC_ConnectX_6', name=nic1_name)
#     c1_scheduler_nic = cloud1.add_component(model='NIC_Basic', name=nic2_name)
    
#     c2_db_nic = cloud2.add_component(model='NIC_ConnectX_6', name=nic3_name)
#     c2_scheduler_nic = cloud2.add_component(model='NIC_Basic', name=nic4_name)
    
#     db1_nic = database1.add_component(model='NIC_ConnectX_5', name=nic5_name)
#     db2_nic = database2.add_component(model='NIC_ConnectX_5', name=nic6_name)
    
#     scheduler_nic1 = scheduler.add_component(model='NIC_Basic', name=nic7_name)
#     scheduler_nic2 = scheduler.add_component(model='NIC_Basic', name=nic8_name)
#     # scheduler_nic = scheduler.add_component(model='NIC_ConnectX_5', name=nic7_name)
    

    
#     #connections
#     bridge1 = slice1.add_l2network(name=bridge1_name, interfaces=[
#         scheduler_nic1.get_interfaces()[0], c1_scheduler_nic.get_interfaces()[0]])
#     bridge2 = slice1.add_l2network(name=bridge2_name, interfaces=[
#         scheduler_nic2.get_interfaces()[0], c2_scheduler_nic.get_interfaces()[0]])
    
#     bridge3 = slice1.add_l2network(name=bridge3_name, interfaces=[
#         c1_db_nic.get_interfaces()[0], db1_nic.get_interfaces()[0]])
#     bridge4 = slice1.add_l2network(name=bridge4_name, interfaces=[
#         c1_db_nic.get_interfaces()[1], db2_nic.get_interfaces()[0]])
    
#     bridge5 = slice1.add_l2network(name=bridge5_name, interfaces=[
#         c2_db_nic.get_interfaces()[0], db1_nic.get_interfaces()[1]])
#     bridge6 = slice1.add_l2network(name=bridge6_name, interfaces=[
#         c2_db_nic.get_interfaces()[1], db2_nic.get_interfaces()[1]])
    
    
    c1_db1_nic = cloud1.add_component(model='NIC_Basic', name='c1_db1_nic')
    c1_db2_nic = cloud1.add_component(model='NIC_Basic', name='c1_db2_nic')
    c1_scheduler_nic = cloud1.add_component(model='NIC_Basic', name='c1_scheduler_nic')
    
    c2_db1_nic = cloud2.add_component(model='NIC_Basic', name='c2_db1_nic')
    c2_db2_nic = cloud2.add_component(model='NIC_Basic', name='c2_db2_nic')
    c2_scheduler_nic = cloud2.add_component(model='NIC_Basic', name='c2_scheduler_nic')
    
    db1_c1_nic = database1.add_component(model='NIC_Basic', name='db1_c1_nic')
    db1_c2_nic = database1.add_component(model='NIC_Basic', name='db1_c2_nic')
    db2_c1_nic = database2.add_component(model='NIC_Basic', name='db2_c1_nic')
    db2_c2_nic = database2.add_component(model='NIC_Basic', name='db2_c2_nic')
    
    scheduler_nic1 = scheduler.add_component(model='NIC_Basic', name='scheduler_nic1')
    scheduler_nic2 = scheduler.add_component(model='NIC_Basic', name='scheduler_nic2')
    # scheduler_nic = scheduler.add_component(model='NIC_ConnectX_5', name=nic7_name)
    

    
    #connections
    bridge1 = slice1.add_l2network(name='bridge_c1_scheduler', interfaces=[
        scheduler_nic1.get_interfaces()[0], c1_scheduler_nic.get_interfaces()[0]])
    bridge2 = slice1.add_l2network(name='bridge_c2_scheduler', interfaces=[
        scheduler_nic2.get_interfaces()[0], c2_scheduler_nic.get_interfaces()[0]])
    
    bridge3 = slice1.add_l2network(name='bridge_c1_db1', interfaces=[
        c1_db1_nic.get_interfaces()[0], db1_c1_nic.get_interfaces()[0]])
    bridge4 = slice1.add_l2network(name='bridge_c1_db2', interfaces=[
        c1_db2_nic.get_interfaces()[0], db2_c1_nic.get_interfaces()[0]])
    
    bridge5 = slice1.add_l2network(name='bridge_c2_db1', interfaces=[
        c2_db1_nic.get_interfaces()[0], db1_c2_nic.get_interfaces()[0]])
    bridge6 = slice1.add_l2network(name='bridge_c2_db2', interfaces=[
        c2_db2_nic.get_interfaces()[0], db2_c2_nic.get_interfaces()[0]])
    
    #Submit Slice Request
    slice1.submit(wait_timeout=3000)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()


Retry: 4, Time: 291 sec


ID,39c7885e-d6c2-4936-89a7-3d9b13fb69a0
Name,MultiCloud2
Lease Expiration (UTC),2023-05-10 23:44:57 +0000
Lease Start (UTC),2023-05-09 23:44:58 +0000
Project ID,1630021f-0a0c-4792-a241-997f410d36e1
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
bc2fae14-82bb-40aa-a520-6f063535e098,scheduler_node,2,4,10,default_ubuntu_20,qcow2,tacc-w3.fabric-testbed.net,TACC,ubuntu,129.114.110.84,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1ac2c19b-713f-4435-b82e-5874a6ddb1ae,cloud1_node,4,16,10,default_ubuntu_20,qcow2,tacc-w3.fabric-testbed.net,TACC,ubuntu,129.114.110.98,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
4301565b-0f26-4b8c-bcd2-76058f173cd3,cloud2_node,4,16,10,default_ubuntu_20,qcow2,tacc-w3.fabric-testbed.net,TACC,ubuntu,129.114.110.95,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
fd898fbf-25c2-4efb-a753-9fe3ca020590,database1_node,2,4,10,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.105,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
b54334cc-c49f-4f02-aab5-0fb2c55d9944,database2_node,2,4,10,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.103,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
445948c0-940c-4100-89e6-e48fe1d372c3,bridge_c1_scheduler,L2,L2Bridge,TACC,None,None,Active,
f11e8b8a-86f2-4189-a2ae-806d4077621a,bridge_c2_scheduler,L2,L2Bridge,TACC,None,None,Active,
68bcec39-0192-41b9-8f9b-c47d7be87d7a,bridge_c1_db1,L2,L2Bridge,TACC,None,None,Active,
78926bde-1f6f-46d1-83ce-ef5bce49d2b5,bridge_c1_db2,L2,L2Bridge,TACC,None,None,Active,
88ff7a44-4b02-4a3f-8b7e-3e2af790a52b,bridge_c2_db1,L2,L2Bridge,TACC,None,None,Active,
1ad43db5-65b1-4d4f-8260-1141e0cf7081,bridge_c2_db2,L2,L2Bridge,TACC,None,None,Active,



Time to stable 291 seconds
Running post_boot_config ... Time to post boot config 330 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
scheduler_node-scheduler_nic1-p1,scheduler_node,bridge_c1_scheduler,100,None,06:74:B5:8E:F5:55,ens8,ens8
scheduler_node-scheduler_nic2-p1,scheduler_node,bridge_c2_scheduler,100,None,06:3C:90:EC:8B:3A,ens7,ens7
cloud1_node-c1_db2_nic-p1,cloud1_node,bridge_c1_db2,100,None,0A:9D:06:6C:0D:26,ens8,ens8
cloud1_node-c1_scheduler_nic-p1,cloud1_node,bridge_c1_scheduler,100,None,0A:F2:C7:BD:BB:56,ens9,ens9
cloud1_node-c1_db1_nic-p1,cloud1_node,bridge_c1_db1,100,None,0A:12:DC:4C:3E:AA,ens7,ens7
cloud2_node-c2_db2_nic-p1,cloud2_node,bridge_c2_db2,100,None,0E:46:FE:74:94:EC,ens7,ens7
cloud2_node-c2_scheduler_nic-p1,cloud2_node,bridge_c2_scheduler,100,None,0E:AC:6B:3B:4C:96,ens8,ens8
cloud2_node-c2_db1_nic-p1,cloud2_node,bridge_c2_db1,100,None,12:77:27:2B:5D:74,ens9,ens9
database1_node-db1_c1_nic-p1,database1_node,bridge_c1_db1,100,None,06:09:BC:72:F3:EF,ens8,ens8
database1_node-db1_c2_nic-p1,database1_node,bridge_c2_db1,100,None,02:EE:00:C6:75:96,ens7,ens7



Time to print interfaces 352 seconds


In [18]:
slice1 = fablib.get_slices()
print(slice1)

In [3]:
slice1 = fablib.get_slice(name=slice_name)
print(slice1)

-----------  ------------------------------------
Slice Name   MultiCloud2
Slice ID     39c7885e-d6c2-4936-89a7-3d9b13fb69a0
Slice State  StableOK
Lease End    2023-05-15 00:02:00 +0000
-----------  ------------------------------------


In [6]:
# fablib.get_slice(name=slice_name).delete()

In [20]:
try:
    slice1 = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice1.get_lease_end()}")
       
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Lease End         : 2023-05-10 23:44:57 +0000


In [28]:
import datetime

end_date = datetime.datetime.now()
end_date = end_date.replace(tzinfo=datetime.timezone.utc)
end_date = (end_date + datetime.timedelta(days=5)).strftime("%Y-%m-%d %H:%M:%S %z")
print(end_date)

try:
    slice1 = fablib.get_slice(name=slice_name)
    slice1.renew(end_date)
except Exception as e:
    print(f"Get Slices Fail: {e}")
    traceback.print_exc()

2023-05-15 00:02:51 +0000
Get Slices Fail: Failed to renew slice: Status.FAILURE, (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Wed, 10 May 2023 00:02:52 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '240', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'DNT, User-Agent, X-Requested-With, If-Modified-Since, Cache-Control, Content-Type, Range, Authorization', 'Access-Control-Allow-Methods': 'GET, POST, PUT, PATCH, DELETE, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, Content-Range, X-Error', 'X-Error': 'Attempted new term end time is shorter than current slice end time'})
HTTP response body: b'{\n    "errors": [\n        {\n            "details": "Attempted new term end time is shorter than current slice end time",\n            "message": "Internal Server Error"\n        }\n    ],\n    "size": 1,\

Traceback (most recent call last):
  File "/tmp/ipykernel_143/1009903522.py", line 10, in <module>
    slice1.renew(end_date)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 1296, in renew
    raise Exception(
Exception: Failed to renew slice: Status.FAILURE, (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Wed, 10 May 2023 00:02:52 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '240', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'DNT, User-Agent, X-Requested-With, If-Modified-Since, Cache-Control, Content-Type, Range, Authorization', 'Access-Control-Allow-Methods': 'GET, POST, PUT, PATCH, DELETE, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, Content-Range, X-Error', 'X-Error': 'Attempted new term end time is shorter than current slice end time'})
HTTP

In [29]:
import datetime

end_date = datetime.datetime.now()
end_date = end_date.replace(tzinfo=datetime.timezone.utc)
end_date = (end_date + datetime.timedelta(days=12)).strftime("%Y-%m-%d %H:%M:%S %z")
print(end_date)

try:
    slice1 = fablib.get_slice(name=slice_name)
    slice1.renew(end_date)
except Exception as e:
    print(f"Get Slices Fail: {e}")
    traceback.print_exc()

2023-05-22 00:02:55 +0000
Get Slices Fail: Failed to renew slice: Status.FAILURE, (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Wed, 10 May 2023 00:02:56 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '324', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'DNT, User-Agent, X-Requested-With, If-Modified-Since, Cache-Control, Content-Type, Range, Authorization', 'Access-Control-Allow-Methods': 'GET, POST, PUT, PATCH, DELETE, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, Content-Range, X-Error', 'X-Error': "Failed to extend reservation# ['f11e8b8a-86f2-4189-a2ae-806d4077621a', '445948c0-940c-4100-89e6-e48fe1d372c3', '1ad43db5-65b1-4d4f-8260-1141e0cf7081']"})
HTTP response body: b'{\n    "errors": [\n        {\n            "details": "Failed to extend reservation# [\'f11e8b8a-86f2-4189-a2ae-806d4077621

Traceback (most recent call last):
  File "/tmp/ipykernel_143/1660690637.py", line 10, in <module>
    slice1.renew(end_date)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 1296, in renew
    raise Exception(
Exception: Failed to renew slice: Status.FAILURE, (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Wed, 10 May 2023 00:02:56 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '324', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'DNT, User-Agent, X-Requested-With, If-Modified-Since, Cache-Control, Content-Type, Range, Authorization', 'Access-Control-Allow-Methods': 'GET, POST, PUT, PATCH, DELETE, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, Content-Range, X-Error', 'X-Error': "Failed to extend reservation# ['f11e8b8a-86f2-4189-a2ae-806d4077621a', '44

In [30]:
try:
    slice1 = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice1.get_lease_end()}")
       
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Lease End         : 2023-05-15 00:02:00 +0000


In [31]:
try:
    slice1 = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice1.get_lease_end()}")
       
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Lease End         : 2023-05-15 00:02:00 +0000


In [32]:
try:
    slice1 = fablib.get_slice(name=slice_name)
    for node in slice1.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   Components        : {node.get_components()}")
        print(f"   Interfaces        : {node.get_interfaces()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : scheduler_node
   Cores             : 2
   RAM               : 4
   Disk              : 10
   Image             : default_ubuntu_20
   Image Type        : qcow2
   Host              : tacc-w3.fabric-testbed.net
   Site              : TACC
   Management IP     : 129.114.110.84
   Reservation ID    : bc2fae14-82bb-40aa-a520-6f063535e098
   Reservation State : Active
   Components        : [<fabrictestbed_extensions.fablib.component.Component object at 0x7fcb64728730>, <fabrictestbed_extensions.fablib.component.Component object at 0x7fcb64728c40>]
   Interfaces        : [<fabrictestbed_extensions.fablib.interface.Interface object at 0x7fcb64728430>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7fcb64728a00>]
   SSH Command       : ssh ${Username}@${Management IP}

Node:
   Name              : cloud1_node
   Cores             : 4
   RAM               : 16
   Disk              : 10
   Image             : default_ubuntu_20
   Image Type       

In [33]:
try:
    slice1 = fablib.get_slice(name=slice_name)
    for node in slice1.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   Components        : {node.get_components()}")
        print(f"   Interfaces        : {node.get_interfaces()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : scheduler_node
   Cores             : 2
   RAM               : 4
   Disk              : 10
   Image             : default_ubuntu_20
   Image Type        : qcow2
   Host              : tacc-w3.fabric-testbed.net
   Site              : TACC
   Management IP     : 129.114.110.84
   Reservation ID    : bc2fae14-82bb-40aa-a520-6f063535e098
   Reservation State : Active
   Components        : [<fabrictestbed_extensions.fablib.component.Component object at 0x7fcb842d3b80>, <fabrictestbed_extensions.fablib.component.Component object at 0x7fcb6441db50>]
   Interfaces        : [<fabrictestbed_extensions.fablib.interface.Interface object at 0x7fcb6441d7f0>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7fcb6441dd00>]
   SSH Command       : ssh ${Username}@${Management IP}

Node:
   Name              : cloud1_node
   Cores             : 4
   RAM               : 16
   Disk              : 10
   Image             : default_ubuntu_20
   Image Type       

s - c1 : 192.168.10.1 - 192.168.10.3 | bridge_c1_scheduler

s - c2 : 192.168.10.2 - 192.168.10.6 | bridge_c2_scheduler

c1 - db1 : 192.168.10.4 - 192.168.10.9 | bridge_c1_db1

c1 - db2 : 192.168.10.5 - 192.168.10.11 | bridge_c1_db2

c2 - db1 : 192.168.10.7 - 192.168.10.10 | bridge_c2_db1

c2 - db2 : 192.168.10.8 - 192.168.10.12 | bridge_c2_db2

In [4]:
scheduler_node = slice1.get_node("scheduler_node")
cloud1_node = slice1.get_node("cloud1_node")
cloud2_node = slice1.get_node("cloud2_node")
database1_node = slice1.get_node("database1_node")
database2_node = slice1.get_node("database2_node")

In [35]:
cloud1_node.execute("ls -l")

total 0


('total 0\n', '')

In [36]:
scheduler_node.execute("ls -l")

total 0


('total 0\n', '')

### First, we install net-tools in order to be able to configure routes.

In [37]:
scheduler_node.execute("sudo apt update && sudo apt install net-tools")
cloud1_node.execute("sudo apt update && sudo apt install net-tools")
cloud2_node.execute("sudo apt update && sudo apt install net-tools")
database1_node.execute("sudo apt update && sudo apt install net-tools")
database2_node.execute("sudo apt update && sudo apt install net-tools")

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
 

 Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse Translation-en [104 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 c-n-f Metadata [9136 B]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2535 kB]
Get:12 http://nova.clouds.ar

("Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease\nGet:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]\nGet:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]\nGet:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]\nGet:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2152 kB]\nGet:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]\nGet:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [347 kB]\nGet:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [12.6 kB]\nGet:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1719 kB]\nGet:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]\nGet:11 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [242 kB]\nGet:12 http://security.ubuntu.com/ubuntu focal-secu

## IP address configuration and route configuration.

In [38]:
print(scheduler_node.execute("route")[0])

Kernel IP routing table
Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
default         _gateway        0.0.0.0         UG    100    0        0 ens3
10.20.4.0       0.0.0.0         255.255.254.0   U     0      0        0 ens3
169.254.169.254 _gateway        255.255.255.255 UGH   100    0        0 ens3
Kernel IP routing table
Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
default         _gateway        0.0.0.0         UG    100    0        0 ens3
10.20.4.0       0.0.0.0         255.255.254.0   U     0      0        0 ens3
169.254.169.254 _gateway        255.255.255.255 UGH   100    0        0 ens3



In [39]:
print(scheduler_node.execute("ip a")[0])

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: ens3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:21:a9:61 brd ff:ff:ff:ff:ff:ff
    inet 10.20.4.91/23 brd 10.20.5.255 scope global dynamic ens3
       valid_lft 85169sec preferred_lft 85169sec
    inet6 fe80::f816:3eff:fe21:a961/64 scope link 
       valid_lft forever preferred_lft forever
3: ens7: <BROADCAST,MULTICAST> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether 06:3c:90:ec:8b:3a brd ff:ff:ff:ff:ff:ff
4: ens8: <BROADCAST,MULTICAST> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether 06:74:b5:8e:f5:55 brd ff:ff:ff:ff:ff:ff
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qd

In [81]:
# bridge1_name ='bridge_c1_scheduler'
# bridge2_name ='bridge_c2_scheduler'
# bridge3_name ='bridge_c1_db1'
# bridge4_name ='bridge_c1_db2'
# bridge5_name ='bridge_c2_db1'
# bridge6_name ='bridge_c2_db2'

In [40]:
scheduler_node.get_interface(network_name="bridge_c1_scheduler").set_ip(ip="192.168.10.1", cidr="32")
scheduler_node.get_interface(network_name="bridge_c2_scheduler").set_ip(ip="192.168.10.2", cidr="32")

cloud1_node.get_interface(network_name="bridge_c1_scheduler").set_ip(ip="192.168.10.3", cidr="32")
cloud1_node.get_interface(network_name="bridge_c1_db1").set_ip(ip="192.168.10.4", cidr="32")
cloud1_node.get_interface(network_name="bridge_c1_db2").set_ip(ip="192.168.10.5", cidr="32")

cloud2_node.get_interface(network_name="bridge_c2_scheduler").set_ip(ip="192.168.10.6", cidr="32")
cloud2_node.get_interface(network_name="bridge_c2_db1").set_ip(ip="192.168.10.7", cidr="32")
cloud2_node.get_interface(network_name="bridge_c2_db2").set_ip(ip="192.168.10.8", cidr="32")

database1_node.get_interface(network_name="bridge_c1_db1").set_ip(ip="192.168.10.9", cidr="32")
database1_node.get_interface(network_name="bridge_c2_db1").set_ip(ip="192.168.10.10", cidr="32")

database2_node.get_interface(network_name="bridge_c1_db2").set_ip(ip="192.168.10.11", cidr="32")
database2_node.get_interface(network_name="bridge_c2_db2").set_ip(ip="192.168.10.12", cidr="32")

In [41]:
scheduler_node.execute("sudo route add 192.168.10.3/32 dev " + scheduler_node.get_interface(network_name="bridge_c1_scheduler").get_os_interface())

('', '')

In [42]:
scheduler_node.execute("sudo route add 192.168.10.6/32 dev " + scheduler_node.get_interface(network_name="bridge_c2_scheduler").get_os_interface())

('', '')

In [43]:
cloud1_node.execute("sudo route add 192.168.10.1/32 dev " + cloud1_node.get_interface(network_name="bridge_c1_scheduler").get_os_interface())

('', '')

In [44]:
cloud2_node.execute("sudo route add 192.168.10.2/32 dev " + cloud2_node.get_interface(network_name="bridge_c2_scheduler").get_os_interface())

('', '')

In [45]:
cloud1_node.execute("sudo route add 192.168.10.9/32 dev " + cloud1_node.get_interface(network_name="bridge_c1_db1").get_os_interface())

('', '')

In [46]:
database1_node.execute("sudo route add 192.168.10.4/32 dev " + database1_node.get_interface(network_name="bridge_c1_db1").get_os_interface())

('', '')

In [47]:
cloud1_node.execute("sudo route add 192.168.10.11/32 dev " + cloud1_node.get_interface(network_name="bridge_c1_db2").get_os_interface())

('', '')

In [48]:
database2_node.execute("sudo route add 192.168.10.5/32 dev " + database2_node.get_interface(network_name="bridge_c1_db2").get_os_interface())

('', '')

In [49]:
cloud2_node.execute("sudo route add 192.168.10.10/32 dev " + cloud2_node.get_interface(network_name="bridge_c2_db1").get_os_interface())

('', '')

In [50]:
database1_node.execute("sudo route add 192.168.10.7/32 dev " + database1_node.get_interface(network_name="bridge_c2_db1").get_os_interface())

('', '')

In [51]:
cloud2_node.execute("sudo route add 192.168.10.12/32 dev " + cloud2_node.get_interface(network_name="bridge_c2_db2").get_os_interface())

('', '')

In [52]:
database2_node.execute("sudo route add 192.168.10.8/32 dev " + database2_node.get_interface(network_name="bridge_c2_db2").get_os_interface())

('', '')

## Script and File Uploads.

In [53]:
# file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/our_scheduler.py','our_scheduler.py')
# print(file_attributes)
# file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/our_scheduler_adaptive.py','our_scheduler_adaptive.py')
# print(file_attributes)
# file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/profiling_only.py','profiling_only.py')
# print(file_attributes)
# file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/round_robin.py','round_robin.py')
# print(file_attributes)
# file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/free_resources_first.py','free_resources_first.py')
# print(file_attributes)
# file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/round_robin_per_function.py','round_robin_per_function.py')
# print(file_attributes)
# file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/queue_module.py','queue_module.py')
# print(file_attributes)
file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/main.py','main.py')
print(file_attributes)

-rw-rw-r--   1 1000     1000        20358 10 May 00:08 ?


In [54]:
file_attributes = cloud1_node.upload_file('MultiCloudSchedulerContentionFablibScripts/cloud1_node/cloud1_receiver.py','cloud1_receiver.py')
print(file_attributes)
file_attributes = cloud1_node.upload_file('MultiCloudSchedulerContentionFablibScripts/cloud1_node/cloud1_a1.py','cloud1_a1.py')
print(file_attributes)
file_attributes = cloud1_node.upload_file('MultiCloudSchedulerContentionFablibScripts/cloud1_node/cloud1_a2.py','cloud1_a2.py')
print(file_attributes)
file_attributes = cloud1_node.upload_file('MultiCloudSchedulerContentionFablibScripts/cloud1_node/standalone.conf','standalone.conf')
print(file_attributes)
file_attributes = cloud1_node.upload_file('MultiCloudSchedulerContentionFablibScripts/cloud1_node/openwhisk_run_command.sh','openwhisk_run_command.sh')
print(file_attributes)

-rw-rw-r--   1 1000     1000          885 10 May 00:08 ?
-rw-rw-r--   1 1000     1000          108 10 May 00:08 ?
-rw-rw-r--   1 1000     1000          109 10 May 00:08 ?
-rw-rw-r--   1 1000     1000         4466 10 May 00:08 ?
-rw-rw-r--   1 1000     1000          108 10 May 00:08 ?


In [55]:
file_attributes = cloud2_node.upload_file('MultiCloudSchedulerContentionFablibScripts/cloud2_node/cloud2_receiver.py','cloud2_receiver.py')
print(file_attributes)
file_attributes = cloud2_node.upload_file('MultiCloudSchedulerContentionFablibScripts/cloud2_node/cloud2_a1.py','cloud2_a1.py')
print(file_attributes)
file_attributes = cloud2_node.upload_file('MultiCloudSchedulerContentionFablibScripts/cloud2_node/cloud2_a2.py','cloud2_a2.py')
print(file_attributes)
file_attributes = cloud2_node.upload_file('MultiCloudSchedulerContentionFablibScripts/cloud2_node/standalone.conf','standalone.conf')
print(file_attributes)

-rw-rw-r--   1 1000     1000          885 10 May 00:08 ?
-rw-rw-r--   1 1000     1000          153 10 May 00:08 ?
-rw-rw-r--   1 1000     1000          154 10 May 00:08 ?
-rw-rw-r--   1 1000     1000         4466 10 May 00:08 ?


In [56]:
file_attributes = database1_node.upload_file('MultiCloudSchedulerContentionFablibScripts/database1_node/create_database.py','create_database.py')
print(file_attributes)
file_attributes = database1_node.upload_file('MultiCloudSchedulerContentionFablibScripts/database1_node/database1_cloud1_server.py','database1_cloud1_server.py')
print(file_attributes)
file_attributes = database1_node.upload_file('MultiCloudSchedulerContentionFablibScripts/database1_node/database1_cloud2_server.py','database1_cloud2_server.py')
print(file_attributes)

-rw-rw-r--   1 1000     1000        76092 10 May 00:08 ?
-rw-rw-r--   1 1000     1000          403 10 May 00:08 ?
-rw-rw-r--   1 1000     1000          404 10 May 00:08 ?


In [57]:
file_attributes = database2_node.upload_file('MultiCloudSchedulerContentionFablibScripts/database2_node/create_database.py','create_database.py')
print(file_attributes)
file_attributes = database2_node.upload_file('MultiCloudSchedulerContentionFablibScripts/database2_node/database2_cloud1_server.py','database2_cloud1_server.py')
print(file_attributes)
file_attributes = database2_node.upload_file('MultiCloudSchedulerContentionFablibScripts/database2_node/database2_cloud2_server.py','database2_cloud2_server.py')
print(file_attributes)

-rw-rw-r--   1 1000     1000        76092 10 May 00:08 ?
-rw-rw-r--   1 1000     1000          404 10 May 00:08 ?
-rw-rw-r--   1 1000     1000          404 10 May 00:08 ?


### Scheduler Setup

In [58]:
scheduler_node.execute('sudo apt-get install -y python3-pip')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9
  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl
  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1
  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0
  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot
  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0
  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1
  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel
  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev
Suggested packages:
  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib
  g++-9-multilib gcc-9-doc gcc-multilib autoconf automake li

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9\n  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl\n  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1\n  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0\n  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot\n  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0\n  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal\n  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1\n  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel\n  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev\nSuggested packages:\n  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib\n  g++-9-multilib gcc-9-doc gcc-multilib au

In [59]:
scheduler_node.execute('pip install requests requests-futures')

('Requirement already satisfied: requests in /usr/lib/python3/dist-packages (2.22.0)\nCollecting requests-futures\n  Downloading requests_futures-1.0.0-py2.py3-none-any.whl (7.4 kB)\nInstalling collected packages: requests-futures\nSuccessfully installed requests-futures-1.0.0\n',
 '')

### Clouds Setup

In [60]:
cloud1_node.execute('sudo apt-get install -y python3-pip')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9
  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl
  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1
  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0
  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot
  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0
  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1
  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel
  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev
Suggested packages:
  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib
  g++-9-multilib gcc-9-doc gcc-multilib autoconf automake li

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9\n  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl\n  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1\n  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0\n  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot\n  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0\n  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal\n  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1\n  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel\n  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev\nSuggested packages:\n  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib\n  g++-9-multilib gcc-9-doc gcc-multilib au

In [61]:
cloud1_node.execute('pip install flask requests')

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


('Collecting flask\n  Downloading Flask-2.3.2-py3-none-any.whl (96 kB)\nRequirement already satisfied: requests in /usr/lib/python3/dist-packages (2.22.0)\nCollecting Jinja2>=3.1.2\n  Downloading Jinja2-3.1.2-py3-none-any.whl (133 kB)\nCollecting importlib-metadata>=3.6.0; python_version < "3.10"\n  Downloading importlib_metadata-6.6.0-py3-none-any.whl (22 kB)\nCollecting Werkzeug>=2.3.3\n  Downloading Werkzeug-2.3.4-py3-none-any.whl (242 kB)\nCollecting itsdangerous>=2.1.2\n  Downloading itsdangerous-2.1.2-py3-none-any.whl (15 kB)\nCollecting click>=8.1.3\n  Downloading click-8.1.3-py3-none-any.whl (96 kB)\nCollecting blinker>=1.6.2\n  Downloading blinker-1.6.2-py3-none-any.whl (13 kB)\nCollecting MarkupSafe>=2.0\n  Downloading MarkupSafe-2.1.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)\nRequirement already satisfied: zipp>=0.5 in /usr/lib/python3/dist-packages (from importlib-metadata>=3.6.0; python_version < "3.10"->flask) (1.0.0)\nInstalling collected packages

In [62]:
cloud1_node.execute('git clone https://github.com/apache/openwhisk.git')

 Cloning into 'openwhisk'...
 

('', "Cloning into 'openwhisk'...\n")

In [63]:
cloud1_node.execute('sudo apt update && sudo apt install -y default-jre default-jdk nodejs')

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
 

 Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Reading package lists...
Building dependency tree...
Reading state information...
196 packages can be upgraded. Run 'apt list --upgradable' to see them.
 

 Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  at-spi2-core ca-certificates-java default-jdk-headless default-jre-headless
  fontconfig-config fonts-dejavu-core fonts-dejavu-extra java-common
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libavahi-client3 libavahi-common-data
  libavahi-common3 libc-ares2 libcups2 libdrm-amdgpu1 libdrm-intel1
  libdrm-nouveau2 libdrm-radeon1 libfontconfig1 libfontenc1

("Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease\nHit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease\nHit:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease\nHit:4 http://security.ubuntu.com/ubuntu focal-security InRelease\nReading package lists...\nBuilding dependency tree...\nReading state information...\n196 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  at-spi2-core ca-certificates-java default-jdk-headless default-jre-headless\n  fontconfig-config fonts-dejavu-core fonts-dejavu-extra java-common\n  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0\n  libatk1.0-data libatspi2.0-0 libavahi-client3 libavahi-common-data\n  libavahi-common3 libc-ares2 libcups2 libdrm-amdgpu1 libdrm-intel1\n  libdrm-nouveau2 libdrm-radeon1 libfontconfig1 

In [64]:
cloud1_node.execute('curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg')

('', '')

In [65]:
cloud1_node.execute('echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null')

('', '')

In [66]:
cloud1_node.execute('sudo apt update && sudo apt install -y docker-ce docker-ce-cli containerd.io')

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
 

 Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Get:4 https://download.docker.com/linux/ubuntu focal InRelease [57.7 kB]
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease
Get:6 https://download.docker.com/linux/ubuntu focal/stable amd64 Packages [28.6 kB]
Fetched 86.2 kB in 0s (186 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
196 packages can be upgraded. Run 'apt list --upgradable' to see them.
 

 Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  docker-buildx-plugin docker-ce-rootless-extras docker-compose-plugin pigz
  slirp4netns
Suggested packages:
  aufs-tools cgroupfs-mount | cgroup-lite
The following NEW packages will be installed:
  containerd.io docker-buildx-p

("Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease\nHit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease\nHit:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease\nGet:4 https://download.docker.com/linux/ubuntu focal InRelease [57.7 kB]\nHit:5 http://security.ubuntu.com/ubuntu focal-security InRelease\nGet:6 https://download.docker.com/linux/ubuntu focal/stable amd64 Packages [28.6 kB]\nFetched 86.2 kB in 0s (186 kB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\n196 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  docker-buildx-plugin docker-ce-rootless-extras docker-compose-plugin pigz\n  slirp4netns\nSuggested packages:\n  aufs-tools cgroupfs-mount | cgroup-lite\nThe following NEW packages will be installed:\n  containerd.io d

In [67]:
cloud1_node.execute('sudo apt install -y npm')

Reading package lists...
 

 Building dependency tree...
Reading state information...
The following additional packages will be installed:
  gyp javascript-common libauthen-sasl-perl libdata-dump-perl
  libencode-locale-perl libfile-basedir-perl libfile-desktopentry-perl
  libfile-listing-perl libfile-mimeinfo-perl libfont-afm-perl
  libhtml-form-perl libhtml-format-perl libhtml-parser-perl
  libhtml-tagset-perl libhtml-tree-perl libhttp-cookies-perl
  libhttp-daemon-perl libhttp-date-perl libhttp-message-perl
  libhttp-negotiate-perl libio-html-perl libio-socket-ssl-perl
  libio-stringy-perl libipc-system-simple-perl libjs-inherits
  libjs-is-typedarray libjs-psl libjs-typedarray-to-buffer
  liblwp-mediatypes-perl liblwp-protocol-https-perl libmailtools-perl
  libnet-dbus-perl libnet-http-perl libnet-smtp-ssl-perl libnet-ssleay-perl
  libnode-dev libpython2-stdlib libpython2.7-minimal libpython2.7-stdlib
  libssl-dev libssl1.1 libtie-ixhash-perl libtimedate-perl libtry-tiny-perl
  lib

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  gyp javascript-common libauthen-sasl-perl libdata-dump-perl\n  libencode-locale-perl libfile-basedir-perl libfile-desktopentry-perl\n  libfile-listing-perl libfile-mimeinfo-perl libfont-afm-perl\n  libhtml-form-perl libhtml-format-perl libhtml-parser-perl\n  libhtml-tagset-perl libhtml-tree-perl libhttp-cookies-perl\n  libhttp-daemon-perl libhttp-date-perl libhttp-message-perl\n  libhttp-negotiate-perl libio-html-perl libio-socket-ssl-perl\n  libio-stringy-perl libipc-system-simple-perl libjs-inherits\n  libjs-is-typedarray libjs-psl libjs-typedarray-to-buffer\n  liblwp-mediatypes-perl liblwp-protocol-https-perl libmailtools-perl\n  libnet-dbus-perl libnet-http-perl libnet-smtp-ssl-perl libnet-ssleay-perl\n  libnode-dev libpython2-stdlib libpython2.7-minimal libpython2.7-stdlib\n  libssl-dev libssl1.1 libtie-ixhash-perl libtimedate-perl libtry-t

In [68]:
cloud1_node.execute('curl https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh --output homebrew_install.sh')

   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30183  100 30183    0     0   256k      0 --:--:-- --:--:-- --:--:--  258k
 

('',
 '  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\r  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r100 30183  100 30183    0     0   256k      0 --:--:-- --:--:-- --:--:--  258k\n')

In [69]:
cloud1_node.execute('chmod +x homebrew_install.sh')

('', '')

In [70]:
cloud1_node.execute('sudo -u ubuntu ./homebrew_install.sh')

==> Checking for `sudo` access (which may request your password)...
 ==> This script will install:
/home/linuxbrew/.linuxbrew/bin/brew
/home/linuxbrew/.linuxbrew/share/doc/homebrew
/home/linuxbrew/.linuxbrew/share/man/man1/brew.1
/home/linuxbrew/.linuxbrew/share/zsh/site-functions/_brew
/home/linuxbrew/.linuxbrew/etc/bash_completion.d/brew
/home/linuxbrew/.linuxbrew/Homebrew
==> The following new directories will be created:
/home/linuxbrew/.linuxbrew/bin
/home/linuxbrew/.linuxbrew/etc
/home/linuxbrew/.linuxbrew/include
/home/linuxbrew/.linuxbrew/lib
/home/linuxbrew/.linuxbrew/sbin
/home/linuxbrew/.linuxbrew/share
/home/linuxbrew/.linuxbrew/var
/home/linuxbrew/.linuxbrew/opt
/home/linuxbrew/.linuxbrew/share/zsh
/home/linuxbrew/.linuxbrew/share/zsh/site-functions
/home/linuxbrew/.linuxbrew/var/homebrew
/home/linuxbrew/.linuxbrew/var/homebrew/linked
/home/linuxbrew/.linuxbrew/Cellar
/home/linuxbrew/.linuxbrew/Caskroom
/home/linuxbrew/.linuxbrew/Frameworks
==> /usr/bin/sudo /usr/bin/insta

('==> Checking for `sudo` access (which may request your password)...\n==> This script will install:\n/home/linuxbrew/.linuxbrew/bin/brew\n/home/linuxbrew/.linuxbrew/share/doc/homebrew\n/home/linuxbrew/.linuxbrew/share/man/man1/brew.1\n/home/linuxbrew/.linuxbrew/share/zsh/site-functions/_brew\n/home/linuxbrew/.linuxbrew/etc/bash_completion.d/brew\n/home/linuxbrew/.linuxbrew/Homebrew\n==> The following new directories will be created:\n/home/linuxbrew/.linuxbrew/bin\n/home/linuxbrew/.linuxbrew/etc\n/home/linuxbrew/.linuxbrew/include\n/home/linuxbrew/.linuxbrew/lib\n/home/linuxbrew/.linuxbrew/sbin\n/home/linuxbrew/.linuxbrew/share\n/home/linuxbrew/.linuxbrew/var\n/home/linuxbrew/.linuxbrew/opt\n/home/linuxbrew/.linuxbrew/share/zsh\n/home/linuxbrew/.linuxbrew/share/zsh/site-functions\n/home/linuxbrew/.linuxbrew/var/homebrew\n/home/linuxbrew/.linuxbrew/var/homebrew/linked\n/home/linuxbrew/.linuxbrew/Cellar\n/home/linuxbrew/.linuxbrew/Caskroom\n/home/linuxbrew/.linuxbrew/Frameworks\n==> /us

In [71]:
cloud1_node.execute("sudo -u ubuntu /home/linuxbrew/.linuxbrew/bin/brew install wsk") #http_proxy=161.35.125.167 && https_proxy=161.35.125.167 &&

==> Fetching dependencies for wsk: linux-headers@5.15, glibc, gmp, isl, mpfr, libmpc, lz4, xz, zlib, zstd, binutils and gcc
==> Fetching linux-headers@5.15
==> Downloading https://ghcr.io/v2/homebrew/core/linux-headers/5.15/manifests/5.15.110
==> Downloading https://ghcr.io/v2/homebrew/core/linux-headers/5.15/blobs/sha256:41bc8342d481bb3af8b780aa27fa3264d217d55466c8b6cb708ff79613a5f848
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sha256:41bc8342d481bb3af8b780aa27fa3264d217d55466c8b6cb708ff79613a5f848?se=2023-05-10T00%3A20%3A00Z&sig=07ZEDx94%2Fq3IntzDsdqbV19aC9elSPtDdQE203lJGhg%3D&sp=r&spr=https&sr=b&sv=2019-12-12
==> Fetching glibc
==> Downloading https://ghcr.io/v2/homebrew/core/glibc/manifests/2.35_1
==> Downloading https://ghcr.io/v2/homebrew/core/glibc/blobs/sha256:274dd06ae6ecaee3025d6bf21cf4c7641df9a1cc3973e162911a1f4a76000a24
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sha256:274dd06ae6ecaee3025d6bf21cf4c7641df9a

('==> Fetching dependencies for wsk: linux-headers@5.15, glibc, gmp, isl, mpfr, libmpc, lz4, xz, zlib, zstd, binutils and gcc\n==> Fetching linux-headers@5.15\n==> Downloading https://ghcr.io/v2/homebrew/core/linux-headers/5.15/manifests/5.15.110\n==> Downloading https://ghcr.io/v2/homebrew/core/linux-headers/5.15/blobs/sha256:41bc8342d481bb3af8b780aa27fa3264d217d55466c8b6cb708ff79613a5f848\n==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sha256:41bc8342d481bb3af8b780aa27fa3264d217d55466c8b6cb708ff79613a5f848?se=2023-05-10T00%3A20%3A00Z&sig=07ZEDx94%2Fq3IntzDsdqbV19aC9elSPtDdQE203lJGhg%3D&sp=r&spr=https&sr=b&sv=2019-12-12\n==> Fetching glibc\n==> Downloading https://ghcr.io/v2/homebrew/core/glibc/manifests/2.35_1\n==> Downloading https://ghcr.io/v2/homebrew/core/glibc/blobs/sha256:274dd06ae6ecaee3025d6bf21cf4c7641df9a1cc3973e162911a1f4a76000a24\n==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sha256:274dd06ae6ecaee3025d6bf21cf

---

In [72]:
cloud2_node.execute('sudo apt-get install -y python3-pip')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9
  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl
  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1
  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0
  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot
  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0
  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1
  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel
  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev
Suggested packages:
  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib
  g++-9-multilib gcc-9-doc gcc-multilib autoconf automake li

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9\n  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl\n  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1\n  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0\n  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot\n  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0\n  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal\n  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1\n  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel\n  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev\nSuggested packages:\n  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib\n  g++-9-multilib gcc-9-doc gcc-multilib au

In [73]:
cloud2_node.execute('pip install flask requests')

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


('Collecting flask\n  Downloading Flask-2.3.2-py3-none-any.whl (96 kB)\nRequirement already satisfied: requests in /usr/lib/python3/dist-packages (2.22.0)\nCollecting Jinja2>=3.1.2\n  Downloading Jinja2-3.1.2-py3-none-any.whl (133 kB)\nCollecting click>=8.1.3\n  Downloading click-8.1.3-py3-none-any.whl (96 kB)\nCollecting importlib-metadata>=3.6.0; python_version < "3.10"\n  Downloading importlib_metadata-6.6.0-py3-none-any.whl (22 kB)\nCollecting blinker>=1.6.2\n  Downloading blinker-1.6.2-py3-none-any.whl (13 kB)\nCollecting itsdangerous>=2.1.2\n  Downloading itsdangerous-2.1.2-py3-none-any.whl (15 kB)\nCollecting Werkzeug>=2.3.3\n  Downloading Werkzeug-2.3.4-py3-none-any.whl (242 kB)\nCollecting MarkupSafe>=2.0\n  Downloading MarkupSafe-2.1.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)\nRequirement already satisfied: zipp>=0.5 in /usr/lib/python3/dist-packages (from importlib-metadata>=3.6.0; python_version < "3.10"->flask) (1.0.0)\nInstalling collected packages

In [74]:
cloud2_node.execute('git clone https://github.com/apache/openwhisk.git')

 Cloning into 'openwhisk'...
 

('', "Cloning into 'openwhisk'...\n")

In [75]:
cloud2_node.execute('sudo apt update && sudo apt install -y default-jre default-jdk nodejs')

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
 

 Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...
Building dependency tree...
Reading state information...
196 packages can be upgraded. Run 'apt list --upgradable' to see them.
 

 Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  at-spi2-core ca-certificates-java default-jdk-headless default-jre-headless
  fontconfig-config fonts-dejavu-core fonts-dejavu-extra java-common
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libavahi-client3 libavahi-common-data
  libavahi-common3 libc-ares2 libcups2 libdrm-amdgpu1 libdrm-intel1
  libdrm-nouveau2 libdrm-radeon1 libfontconfig1 libfontenc1

("Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease\nHit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease\nHit:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease\nHit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease\nReading package lists...\nBuilding dependency tree...\nReading state information...\n196 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  at-spi2-core ca-certificates-java default-jdk-headless default-jre-headless\n  fontconfig-config fonts-dejavu-core fonts-dejavu-extra java-common\n  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0\n  libatk1.0-data libatspi2.0-0 libavahi-client3 libavahi-common-data\n  libavahi-common3 libc-ares2 libcups2 libdrm-amdgpu1 libdrm-intel1\n  libdrm-nouveau2 libdrm-radeon1 libfontconfig1 

In [76]:
cloud2_node.execute('curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg')

('', '')

In [77]:
cloud2_node.execute('echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null')

('', '')

In [78]:
cloud2_node.execute('sudo apt update && sudo apt install -y docker-ce docker-ce-cli containerd.io')

Get:1 https://download.docker.com/linux/ubuntu focal InRelease [57.7 kB]
 

 Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Get:5 https://download.docker.com/linux/ubuntu focal/stable amd64 Packages [28.6 kB]
Hit:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Fetched 86.2 kB in 1s (153 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
196 packages can be upgraded. Run 'apt list --upgradable' to see them.
 

 
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  docker-buildx-plugin docker-ce-rootless-extras docker-compose-plugin pigz
  slirp4netns
Suggested packages:
  aufs-tools cgroupfs-mount | cgroup-lite
The following NEW packages will be installed:
  containerd.io docker-buildx-plugin docker-ce docker-c

("Get:1 https://download.docker.com/linux/ubuntu focal InRelease [57.7 kB]\nHit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease\nHit:3 http://security.ubuntu.com/ubuntu focal-security InRelease\nHit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease\nGet:5 https://download.docker.com/linux/ubuntu focal/stable amd64 Packages [28.6 kB]\nHit:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease\nFetched 86.2 kB in 1s (153 kB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\n196 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  docker-buildx-plugin docker-ce-rootless-extras docker-compose-plugin pigz\n  slirp4netns\nSuggested packages:\n  aufs-tools cgroupfs-mount | cgroup-lite\nThe following NEW packages will be installed:\n  containerd.io d

In [79]:
cloud2_node.execute('sudo apt install -y npm')

Reading package lists...
 

 Building dependency tree...
Reading state information...
The following additional packages will be installed:
  gyp javascript-common libauthen-sasl-perl libdata-dump-perl
  libencode-locale-perl libfile-basedir-perl libfile-desktopentry-perl
  libfile-listing-perl libfile-mimeinfo-perl libfont-afm-perl
  libhtml-form-perl libhtml-format-perl libhtml-parser-perl
  libhtml-tagset-perl libhtml-tree-perl libhttp-cookies-perl
  libhttp-daemon-perl libhttp-date-perl libhttp-message-perl
  libhttp-negotiate-perl libio-html-perl libio-socket-ssl-perl
  libio-stringy-perl libipc-system-simple-perl libjs-inherits
  libjs-is-typedarray libjs-psl libjs-typedarray-to-buffer
  liblwp-mediatypes-perl liblwp-protocol-https-perl libmailtools-perl
  libnet-dbus-perl libnet-http-perl libnet-smtp-ssl-perl libnet-ssleay-perl
  libnode-dev libpython2-stdlib libpython2.7-minimal libpython2.7-stdlib
  libssl-dev libssl1.1 libtie-ixhash-perl libtimedate-perl libtry-tiny-perl
  lib

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  gyp javascript-common libauthen-sasl-perl libdata-dump-perl\n  libencode-locale-perl libfile-basedir-perl libfile-desktopentry-perl\n  libfile-listing-perl libfile-mimeinfo-perl libfont-afm-perl\n  libhtml-form-perl libhtml-format-perl libhtml-parser-perl\n  libhtml-tagset-perl libhtml-tree-perl libhttp-cookies-perl\n  libhttp-daemon-perl libhttp-date-perl libhttp-message-perl\n  libhttp-negotiate-perl libio-html-perl libio-socket-ssl-perl\n  libio-stringy-perl libipc-system-simple-perl libjs-inherits\n  libjs-is-typedarray libjs-psl libjs-typedarray-to-buffer\n  liblwp-mediatypes-perl liblwp-protocol-https-perl libmailtools-perl\n  libnet-dbus-perl libnet-http-perl libnet-smtp-ssl-perl libnet-ssleay-perl\n  libnode-dev libpython2-stdlib libpython2.7-minimal libpython2.7-stdlib\n  libssl-dev libssl1.1 libtie-ixhash-perl libtimedate-perl libtry-t

In [80]:
cloud2_node.execute('curl https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh --output homebrew_install.sh')

   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30183  100 30183    0     0   286k      0 --:--:-- --:--:-- --:--:--  286k
 

('',
 '  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\r  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r100 30183  100 30183    0     0   286k      0 --:--:-- --:--:-- --:--:--  286k\n')

In [81]:
cloud2_node.execute('chmod +x homebrew_install.sh')

('', '')

In [82]:
cloud2_node.execute('sudo -u ubuntu ./homebrew_install.sh')

==> Checking for `sudo` access (which may request your password)...
 ==> This script will install:
/home/linuxbrew/.linuxbrew/bin/brew
/home/linuxbrew/.linuxbrew/share/doc/homebrew
/home/linuxbrew/.linuxbrew/share/man/man1/brew.1
/home/linuxbrew/.linuxbrew/share/zsh/site-functions/_brew
/home/linuxbrew/.linuxbrew/etc/bash_completion.d/brew
/home/linuxbrew/.linuxbrew/Homebrew
==> The following new directories will be created:
/home/linuxbrew/.linuxbrew/bin
/home/linuxbrew/.linuxbrew/etc
/home/linuxbrew/.linuxbrew/include
/home/linuxbrew/.linuxbrew/lib
/home/linuxbrew/.linuxbrew/sbin
/home/linuxbrew/.linuxbrew/share
/home/linuxbrew/.linuxbrew/var
/home/linuxbrew/.linuxbrew/opt
/home/linuxbrew/.linuxbrew/share/zsh
/home/linuxbrew/.linuxbrew/share/zsh/site-functions
/home/linuxbrew/.linuxbrew/var/homebrew
/home/linuxbrew/.linuxbrew/var/homebrew/linked
/home/linuxbrew/.linuxbrew/Cellar
/home/linuxbrew/.linuxbrew/Caskroom
/home/linuxbrew/.linuxbrew/Frameworks
==> /usr/bin/sudo /usr/bin/insta

('==> Checking for `sudo` access (which may request your password)...\n==> This script will install:\n/home/linuxbrew/.linuxbrew/bin/brew\n/home/linuxbrew/.linuxbrew/share/doc/homebrew\n/home/linuxbrew/.linuxbrew/share/man/man1/brew.1\n/home/linuxbrew/.linuxbrew/share/zsh/site-functions/_brew\n/home/linuxbrew/.linuxbrew/etc/bash_completion.d/brew\n/home/linuxbrew/.linuxbrew/Homebrew\n==> The following new directories will be created:\n/home/linuxbrew/.linuxbrew/bin\n/home/linuxbrew/.linuxbrew/etc\n/home/linuxbrew/.linuxbrew/include\n/home/linuxbrew/.linuxbrew/lib\n/home/linuxbrew/.linuxbrew/sbin\n/home/linuxbrew/.linuxbrew/share\n/home/linuxbrew/.linuxbrew/var\n/home/linuxbrew/.linuxbrew/opt\n/home/linuxbrew/.linuxbrew/share/zsh\n/home/linuxbrew/.linuxbrew/share/zsh/site-functions\n/home/linuxbrew/.linuxbrew/var/homebrew\n/home/linuxbrew/.linuxbrew/var/homebrew/linked\n/home/linuxbrew/.linuxbrew/Cellar\n/home/linuxbrew/.linuxbrew/Caskroom\n/home/linuxbrew/.linuxbrew/Frameworks\n==> /us

In [83]:
cloud2_node.execute("sudo -u ubuntu /home/linuxbrew/.linuxbrew/bin/brew install wsk")

==> Fetching dependencies for wsk: linux-headers@5.15, glibc, gmp, isl, mpfr, libmpc, lz4, xz, zlib, zstd, binutils and gcc
==> Fetching linux-headers@5.15
==> Downloading https://ghcr.io/v2/homebrew/core/linux-headers/5.15/manifests/5.15.110
==> Downloading https://ghcr.io/v2/homebrew/core/linux-headers/5.15/blobs/sha256:41bc8342d481bb3af8b780aa27fa3264d217d55466c8b6cb708ff79613a5f848
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sha256:41bc8342d481bb3af8b780aa27fa3264d217d55466c8b6cb708ff79613a5f848?se=2023-05-10T00%3A25%3A00Z&sig=SiSQk%2FbtPR7xgNTJpT4lf7uw0VsJWY31ZTB0v5b%2BNFI%3D&sp=r&spr=https&sr=b&sv=2019-12-12
==> Fetching glibc
==> Downloading https://ghcr.io/v2/homebrew/core/glibc/manifests/2.35_1
==> Downloading https://ghcr.io/v2/homebrew/core/glibc/blobs/sha256:274dd06ae6ecaee3025d6bf21cf4c7641df9a1cc3973e162911a1f4a76000a24
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sha256:274dd06ae6ecaee3025d6bf21cf4c7641df

('==> Fetching dependencies for wsk: linux-headers@5.15, glibc, gmp, isl, mpfr, libmpc, lz4, xz, zlib, zstd, binutils and gcc\n==> Fetching linux-headers@5.15\n==> Downloading https://ghcr.io/v2/homebrew/core/linux-headers/5.15/manifests/5.15.110\n==> Downloading https://ghcr.io/v2/homebrew/core/linux-headers/5.15/blobs/sha256:41bc8342d481bb3af8b780aa27fa3264d217d55466c8b6cb708ff79613a5f848\n==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sha256:41bc8342d481bb3af8b780aa27fa3264d217d55466c8b6cb708ff79613a5f848?se=2023-05-10T00%3A25%3A00Z&sig=SiSQk%2FbtPR7xgNTJpT4lf7uw0VsJWY31ZTB0v5b%2BNFI%3D&sp=r&spr=https&sr=b&sv=2019-12-12\n==> Fetching glibc\n==> Downloading https://ghcr.io/v2/homebrew/core/glibc/manifests/2.35_1\n==> Downloading https://ghcr.io/v2/homebrew/core/glibc/blobs/sha256:274dd06ae6ecaee3025d6bf21cf4c7641df9a1cc3973e162911a1f4a76000a24\n==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sha256:274dd06ae6ecaee3025d6bf21

### Databases Setup

In [84]:
database1_node.execute('sudo apt-get install -y python3-pip')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9
  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl
  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1
  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0
  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot
  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0
  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1
  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel
  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev
Suggested packages:
  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib
  g++-9-multilib gcc-9-doc gcc-multilib autoconf automake li

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9\n  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl\n  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1\n  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0\n  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot\n  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0\n  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal\n  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1\n  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel\n  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev\nSuggested packages:\n  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib\n  g++-9-multilib gcc-9-doc gcc-multilib au

In [85]:
database1_node.execute('pip install flask')

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


('Collecting flask\n  Downloading Flask-2.3.2-py3-none-any.whl (96 kB)\nCollecting blinker>=1.6.2\n  Downloading blinker-1.6.2-py3-none-any.whl (13 kB)\nCollecting Jinja2>=3.1.2\n  Downloading Jinja2-3.1.2-py3-none-any.whl (133 kB)\nCollecting importlib-metadata>=3.6.0; python_version < "3.10"\n  Downloading importlib_metadata-6.6.0-py3-none-any.whl (22 kB)\nCollecting click>=8.1.3\n  Downloading click-8.1.3-py3-none-any.whl (96 kB)\nCollecting Werkzeug>=2.3.3\n  Downloading Werkzeug-2.3.4-py3-none-any.whl (242 kB)\nCollecting itsdangerous>=2.1.2\n  Downloading itsdangerous-2.1.2-py3-none-any.whl (15 kB)\nCollecting MarkupSafe>=2.0\n  Downloading MarkupSafe-2.1.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)\nRequirement already satisfied: zipp>=0.5 in /usr/lib/python3/dist-packages (from importlib-metadata>=3.6.0; python_version < "3.10"->flask) (1.0.0)\nInstalling collected packages: blinker, MarkupSafe, Jinja2, importlib-metadata, click, Werkzeug, itsdangerous, fl

---

In [86]:
database2_node.execute('sudo apt-get install -y python3-pip')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9
  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl
  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1
  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0
  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot
  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0
  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1
  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel
  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev
Suggested packages:
  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib
  g++-9-multilib gcc-9-doc gcc-multilib autoconf automake li

('Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  binutils binutils-common binutils-x86-64-linux-gnu build-essential cpp cpp-9\n  dpkg-dev fakeroot g++ g++-9 gcc gcc-9 gcc-9-base libalgorithm-diff-perl\n  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1\n  libbinutils libc-dev-bin libc6 libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0\n  libctf0 libdpkg-perl libexpat1 libexpat1-dev libfakeroot\n  libfile-fcntllock-perl libgcc-9-dev libgomp1 libisl22 libitm1 liblsan0\n  libmpc3 libpython3-dev libpython3.8 libpython3.8-dev libpython3.8-minimal\n  libpython3.8-stdlib libquadmath0 libstdc++-9-dev libtsan0 libubsan1\n  linux-libc-dev make manpages-dev python-pip-whl python3-dev python3-wheel\n  python3.8 python3.8-dev python3.8-minimal zlib1g zlib1g-dev\nSuggested packages:\n  binutils-doc cpp-doc gcc-9-locales debian-keyring g++-multilib\n  g++-9-multilib gcc-9-doc gcc-multilib au

In [87]:
database2_node.execute('pip install flask')

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


('Collecting flask\n  Downloading Flask-2.3.2-py3-none-any.whl (96 kB)\nCollecting itsdangerous>=2.1.2\n  Downloading itsdangerous-2.1.2-py3-none-any.whl (15 kB)\nCollecting Werkzeug>=2.3.3\n  Downloading Werkzeug-2.3.4-py3-none-any.whl (242 kB)\nCollecting Jinja2>=3.1.2\n  Downloading Jinja2-3.1.2-py3-none-any.whl (133 kB)\nCollecting blinker>=1.6.2\n  Downloading blinker-1.6.2-py3-none-any.whl (13 kB)\nCollecting click>=8.1.3\n  Downloading click-8.1.3-py3-none-any.whl (96 kB)\nCollecting importlib-metadata>=3.6.0; python_version < "3.10"\n  Downloading importlib_metadata-6.6.0-py3-none-any.whl (22 kB)\nCollecting MarkupSafe>=2.1.1\n  Downloading MarkupSafe-2.1.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)\nRequirement already satisfied: zipp>=0.5 in /usr/lib/python3/dist-packages (from importlib-metadata>=3.6.0; python_version < "3.10"->flask) (1.0.0)\nInstalling collected packages: itsdangerous, MarkupSafe, Werkzeug, Jinja2, blinker, click, importlib-metadata, 

## Scheduler Configuration. Allow it to open more files.

In [89]:
scheduler_node.execute("echo 'root               soft    nofile          65535\nroot               hard    nofile          65535' | sudo tee -a /etc/security/limits.conf")
scheduler_node.execute("echo '*               soft    nofile          65535\n*               hard    nofile          65535' | sudo tee -a /etc/security/limits.conf")

root               soft    nofile          65535
root               hard    nofile          65535
*               soft    nofile          65535
*               hard    nofile          65535


('*               soft    nofile          65535\n*               hard    nofile          65535\n',
 '')

## Now we start to run the infrastructures.

In [90]:
database1_node.execute("tmux new -d 'python3 database1_cloud1_server.py'")

('', '')

In [91]:
database1_node.execute("tmux new -d 'python3 database1_cloud2_server.py'")

('', '')

In [92]:
database2_node.execute("tmux new -d 'python3 database2_cloud1_server.py'")

('', '')

In [93]:
database2_node.execute("tmux new -d 'python3 database2_cloud2_server.py'")

('', '')

---

In [94]:
cloud1_node.execute("tmux new -d 'python3 cloud1_receiver.py'")

('', '')

In [95]:
cloud1_node.execute("tmux new -d 'cd /home/ubuntu/openwhisk && sudo ./gradlew core:standalone:bootRun --args=\"-c /home/ubuntu/standalone.conf\"'")

('', '')

In [96]:
cloud1_node.execute("/home/linuxbrew/.linuxbrew/Cellar/wsk/1.2.0/bin/wsk -i property set --apihost http://172.17.0.1:3233")

ok: whisk API host set to http://172.17.0.1:3233


('ok: whisk API host set to http://172.17.0.1:3233\n', '')

In [97]:
cloud1_node.execute("/home/linuxbrew/.linuxbrew/Cellar/wsk/1.2.0/bin/wsk -i property set --auth 23bc46b1-71f6-4ed5-8c54-816aa4f8c502:123zO3xZCLrMN6v2BKK1dXYFpXlPkccOFqm12CdAsMgRU4VrNZ9lyGVCGuMDGIwP")

ok: whisk auth set. Run 'wsk property get --auth' to see the new value.


("ok: whisk auth set. Run 'wsk property get --auth' to see the new value.\n",
 '')

In [98]:
cloud2_node.execute("tmux new -d 'python3 cloud2_receiver.py'")

('', '')

In [99]:
cloud2_node.execute("tmux new -d 'cd /home/ubuntu/openwhisk && sudo ./gradlew core:standalone:bootRun --args=\"-c /home/ubuntu/standalone.conf\"'")

('', '')

In [100]:
cloud2_node.execute("/home/linuxbrew/.linuxbrew/Cellar/wsk/1.2.0/bin/wsk -i property set --apihost http://172.17.0.1:3233")

ok: whisk API host set to http://172.17.0.1:3233


('ok: whisk API host set to http://172.17.0.1:3233\n', '')

In [101]:
cloud2_node.execute("/home/linuxbrew/.linuxbrew/Cellar/wsk/1.2.0/bin/wsk -i property set --auth 23bc46b1-71f6-4ed5-8c54-816aa4f8c502:123zO3xZCLrMN6v2BKK1dXYFpXlPkccOFqm12CdAsMgRU4VrNZ9lyGVCGuMDGIwP")

ok: whisk auth set. Run 'wsk property get --auth' to see the new value.


("ok: whisk auth set. Run 'wsk property get --auth' to see the new value.\n",
 '')

#### Please wait five minutes before running the two commands below, to make sure openwhisk has started.

In [102]:
cloud1_node.execute("/home/linuxbrew/.linuxbrew/Cellar/wsk/1.2.0/bin/wsk action create a1 cloud1_a1.py")

ok: created action a1


('ok: created action a1\n', '')

In [103]:
cloud1_node.execute("/home/linuxbrew/.linuxbrew/Cellar/wsk/1.2.0/bin/wsk action create a2 cloud1_a2.py")

ok: created action a2


('ok: created action a2\n', '')

In [104]:
cloud2_node.execute("/home/linuxbrew/.linuxbrew/Cellar/wsk/1.2.0/bin/wsk action create a1 cloud2_a1.py")

ok: created action a1


('ok: created action a1\n', '')

In [105]:
cloud2_node.execute("/home/linuxbrew/.linuxbrew/Cellar/wsk/1.2.0/bin/wsk action create a2 cloud2_a2.py")

ok: created action a2


('ok: created action a2\n', '')

----

In [106]:
file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/other_our_scheduler/our_scheduler_adaptive.py','our_scheduler_adaptive.py')
print(file_attributes)
file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/other_our_scheduler/queue_module.py','queue_module.py')
print(file_attributes)
file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/profiling_only.py','profiling_only.py')
print(file_attributes)
file_attributes = scheduler_node.upload_file('MultiCloudSchedulerContentionFablibScripts/scheduler_node/main.py','main.py')
print(file_attributes)

-rw-rw-r--   1 1000     1000        11255 10 May 00:35 ?
-rw-rw-r--   1 1000     1000        13411 10 May 00:35 ?
-rw-rw-r--   1 1000     1000         9251 10 May 00:35 ?
-rw-rw-r--   1 1000     1000        20358 10 May 00:35 ?


In [108]:
file_attributes = scheduler_node.upload_file('main_2023May9_approximately.py','main_2023May9_approximately.py')
print(file_attributes)

-rw-rw-r--   1 1000     1000        20376 10 May 00:49 ?


## x2 Performance Dispersion

_Note that "tc qdisc add dev" might need to be changed to "tc qdisc change dev" if script execution order is changed._

In [9]:
cloud1_node.execute('sudo tc qdisc change dev ' + cloud1_node.get_interface(network_name="bridge_c1_db1").get_os_interface() + ' root tbf rate 1024kbit burst 32kbit latency 50ms')
cloud1_node.execute('sudo tc qdisc change dev ' + cloud1_node.get_interface(network_name="bridge_c1_db2").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
cloud2_node.execute('sudo tc qdisc change dev ' + cloud2_node.get_interface(network_name="bridge_c2_db1").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
cloud2_node.execute('sudo tc qdisc change dev ' + cloud2_node.get_interface(network_name="bridge_c2_db2").get_os_interface() + ' root tbf rate 1024kbit burst 32kbit latency 50ms')
database1_node.execute('sudo tc qdisc change dev ' + database1_node.get_interface(network_name="bridge_c1_db1").get_os_interface() + ' root tbf rate 1024kbit burst 32kbit latency 50ms')
database1_node.execute('sudo tc qdisc change dev ' + database1_node.get_interface(network_name="bridge_c2_db1").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
database2_node.execute('sudo tc qdisc change dev ' + database2_node.get_interface(network_name="bridge_c1_db2").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
database2_node.execute('sudo tc qdisc change dev ' + database2_node.get_interface(network_name="bridge_c2_db2").get_os_interface() + ' root tbf rate 1024kbit burst 32kbit latency 50ms')

('', '')

In [133]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["27.685994970798493"], "Average_Service_Time_round_robin": ["15.145879358172417"], "Completion_Time_round_robin": [774.4025707244873], "Recent_Average_Service_Time_round_robin_per_function": ["25.497785210609436"], "Average_Service_Time_round_robin_per_function": ["13.99199562251568"], "Completion_Time_round_robin_per_function": [712.3744764328003], "Recent_Average_Service_Time_lbcf": ["16.241431510448457"], "Average_Service_Time_lbcf": ["15.326708951830865"], "Completion_Time_lbcf": [778.6501080989838], "Recent_Average_Service_Time_profiling_only": ["20.527017688751222"], "Average_Service_Time_profiling_only": ["14.11012697517872"], "Completion_Time_profiling_only": [715.6847639083862], "Recent_Average_Service_Time_our_scheduler": ["11.465923464298248"], "Average_Service_Time_our_scheduler": ["13.830918514847756"], "Completion_Time_our_scheduler": [698.0407493114471], "o

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["27.685994970798493"], "Average_Service_Time_round_robin": ["15.145879358172417"], "Completion_Time_round_robin": [774.4025707244873], "Recent_Average_Service_Time_round_robin_per_function": ["25.497785210609436"], "Average_Service_Time_round_robin_per_function": ["13.99199562251568"], "Completion_Time_round_robin_per_function": [712.3744764328003], "Recent_Average_Service_Time_lbcf": ["16.241431510448457"], "Average_Service_Time_lbcf": ["15.326708951830865"], "Completion_Time_lbcf": [778.6501080989838], "Recent_Average_Service_Time_profiling_only": ["20.527017688751222"], "Average_Service_Time_profiling_only": ["14.11012697517872"], "Completion_Time_profiling_only": [715.6847639083862], "Recent_Average_Service_Time_our_scheduler": ["11.465923464298248"], "Average_Service_Time_our_scheduler": ["13.830918514847756"], "Completion_Time_our_scheduler": [698.04074931144

In [10]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["27.81156152486801"], "Average_Service_Time_round_robin": ["15.513222743988036"], "Completion_Time_round_robin": [792.9535021781921], "Recent_Average_Service_Time_round_robin_per_function": ["25.381979632377625"], "Average_Service_Time_round_robin_per_function": ["13.980377514958382"], "Completion_Time_round_robin_per_function": [711.8785898685455], "Recent_Average_Service_Time_lbcf": ["16.980079853534697"], "Average_Service_Time_lbcf": ["14.81700963127613"], "Completion_Time_lbcf": [750.5438272953033], "Recent_Average_Service_Time_profiling_only": ["26.038368141651155"], "Average_Service_Time_profiling_only": ["14.386791252613067"], "Completion_Time_profiling_only": [730.9428195953369], "Recent_Average_Service_Time_our_scheduler": ["12.389434146881104"], "Average_Service_Time_our_scheduler": ["13.237221311092377"], "Completion_Time_our_scheduler": [668.3888969421387], "o

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["27.81156152486801"], "Average_Service_Time_round_robin": ["15.513222743988036"], "Completion_Time_round_robin": [792.9535021781921], "Recent_Average_Service_Time_round_robin_per_function": ["25.381979632377625"], "Average_Service_Time_round_robin_per_function": ["13.980377514958382"], "Completion_Time_round_robin_per_function": [711.8785898685455], "Recent_Average_Service_Time_lbcf": ["16.980079853534697"], "Average_Service_Time_lbcf": ["14.81700963127613"], "Completion_Time_lbcf": [750.5438272953033], "Recent_Average_Service_Time_profiling_only": ["26.038368141651155"], "Average_Service_Time_profiling_only": ["14.386791252613067"], "Completion_Time_profiling_only": [730.9428195953369], "Recent_Average_Service_Time_our_scheduler": ["12.389434146881104"], "Average_Service_Time_our_scheduler": ["13.237221311092377"], "Completion_Time_our_scheduler": [668.38889694213

In [11]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["26.05106452703476"], "Average_Service_Time_round_robin": ["15.103995965242387"], "Completion_Time_round_robin": [772.2252774238586], "Recent_Average_Service_Time_round_robin_per_function": ["25.263446485996248"], "Average_Service_Time_round_robin_per_function": ["14.29089435350895"], "Completion_Time_round_robin_per_function": [727.3632688522339], "Recent_Average_Service_Time_lbcf": ["15.3405207157135"], "Average_Service_Time_lbcf": ["15.115064324378967"], "Completion_Time_lbcf": [768.1816074848175], "Recent_Average_Service_Time_profiling_only": ["18.831792688369752"], "Average_Service_Time_profiling_only": ["14.441861746549606"], "Completion_Time_profiling_only": [731.3837289810181], "Recent_Average_Service_Time_our_scheduler": ["11.312145292758942"], "Average_Service_Time_our_scheduler": ["12.44009664785862"], "Completion_Time_our_scheduler": [628.8954129219055], "othe

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["26.05106452703476"], "Average_Service_Time_round_robin": ["15.103995965242387"], "Completion_Time_round_robin": [772.2252774238586], "Recent_Average_Service_Time_round_robin_per_function": ["25.263446485996248"], "Average_Service_Time_round_robin_per_function": ["14.29089435350895"], "Completion_Time_round_robin_per_function": [727.3632688522339], "Recent_Average_Service_Time_lbcf": ["15.3405207157135"], "Average_Service_Time_lbcf": ["15.115064324378967"], "Completion_Time_lbcf": [768.1816074848175], "Recent_Average_Service_Time_profiling_only": ["18.831792688369752"], "Average_Service_Time_profiling_only": ["14.441861746549606"], "Completion_Time_profiling_only": [731.3837289810181], "Recent_Average_Service_Time_our_scheduler": ["11.312145292758942"], "Average_Service_Time_our_scheduler": ["12.44009664785862"], "Completion_Time_our_scheduler": [628.8954129219055]

In [12]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["27.234154677391054"], "Average_Service_Time_round_robin": ["15.410624063014984"], "Completion_Time_round_robin": [787.6461882591248], "Recent_Average_Service_Time_round_robin_per_function": ["25.060973525047302"], "Average_Service_Time_round_robin_per_function": ["13.976637208580971"], "Completion_Time_round_robin_per_function": [712.7597382068634], "Recent_Average_Service_Time_lbcf": ["15.122440314292907"], "Average_Service_Time_lbcf": ["14.7123236951828"], "Completion_Time_lbcf": [742.606772184372], "Recent_Average_Service_Time_profiling_only": ["25.246407687664032"], "Average_Service_Time_profiling_only": ["15.866463209748268"], "Completion_Time_profiling_only": [805.7894582748413], "Recent_Average_Service_Time_our_scheduler": ["11.87836035490036"], "Average_Service_Time_our_scheduler": ["12.65770934844017"], "Completion_Time_our_scheduler": [639.3929443359375], "othe

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["27.234154677391054"], "Average_Service_Time_round_robin": ["15.410624063014984"], "Completion_Time_round_robin": [787.6461882591248], "Recent_Average_Service_Time_round_robin_per_function": ["25.060973525047302"], "Average_Service_Time_round_robin_per_function": ["13.976637208580971"], "Completion_Time_round_robin_per_function": [712.7597382068634], "Recent_Average_Service_Time_lbcf": ["15.122440314292907"], "Average_Service_Time_lbcf": ["14.7123236951828"], "Completion_Time_lbcf": [742.606772184372], "Recent_Average_Service_Time_profiling_only": ["25.246407687664032"], "Average_Service_Time_profiling_only": ["15.866463209748268"], "Completion_Time_profiling_only": [805.7894582748413], "Recent_Average_Service_Time_our_scheduler": ["11.87836035490036"], "Average_Service_Time_our_scheduler": ["12.65770934844017"], "Completion_Time_our_scheduler": [639.3929443359375]

In [13]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["30.43135303258896"], "Average_Service_Time_round_robin": ["15.347558294177055"], "Completion_Time_round_robin": [785.2355532646179], "Recent_Average_Service_Time_round_robin_per_function": ["26.316376745700836"], "Average_Service_Time_round_robin_per_function": ["14.269409460425377"], "Completion_Time_round_robin_per_function": [726.3734467029572], "Recent_Average_Service_Time_lbcf": ["15.144583404064178"], "Average_Service_Time_lbcf": ["15.191838337421418"], "Completion_Time_lbcf": [769.1439583301544], "Recent_Average_Service_Time_profiling_only": ["22.74024988412857"], "Average_Service_Time_profiling_only": ["14.255639310002326"], "Completion_Time_profiling_only": [723.411340713501], "Recent_Average_Service_Time_our_scheduler": ["12.555805289745331"], "Average_Service_Time_our_scheduler": ["12.364351061940193"], "Completion_Time_our_scheduler": [625.5389659404755], "ot

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["30.43135303258896"], "Average_Service_Time_round_robin": ["15.347558294177055"], "Completion_Time_round_robin": [785.2355532646179], "Recent_Average_Service_Time_round_robin_per_function": ["26.316376745700836"], "Average_Service_Time_round_robin_per_function": ["14.269409460425377"], "Completion_Time_round_robin_per_function": [726.3734467029572], "Recent_Average_Service_Time_lbcf": ["15.144583404064178"], "Average_Service_Time_lbcf": ["15.191838337421418"], "Completion_Time_lbcf": [769.1439583301544], "Recent_Average_Service_Time_profiling_only": ["22.74024988412857"], "Average_Service_Time_profiling_only": ["14.255639310002326"], "Completion_Time_profiling_only": [723.411340713501], "Recent_Average_Service_Time_our_scheduler": ["12.555805289745331"], "Average_Service_Time_our_scheduler": ["12.364351061940193"], "Completion_Time_our_scheduler": [625.538965940475

## x3 Performance Dispersion

In [14]:
cloud1_node.execute('sudo tc qdisc change dev ' + cloud1_node.get_interface(network_name="bridge_c1_db1").get_os_interface() + ' root tbf rate 1536kbit burst 32kbit latency 50ms')
cloud1_node.execute('sudo tc qdisc change dev ' + cloud1_node.get_interface(network_name="bridge_c1_db2").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
cloud2_node.execute('sudo tc qdisc change dev ' + cloud2_node.get_interface(network_name="bridge_c2_db2").get_os_interface() + ' root tbf rate 1536kbit burst 32kbit latency 50ms')
cloud2_node.execute('sudo tc qdisc change dev ' + cloud2_node.get_interface(network_name="bridge_c2_db1").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
database1_node.execute('sudo tc qdisc change dev ' + database1_node.get_interface(network_name="bridge_c1_db1").get_os_interface() + ' root tbf rate 1536kbit burst 32kbit latency 50ms')
database1_node.execute('sudo tc qdisc change dev ' + database1_node.get_interface(network_name="bridge_c2_db1").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
database2_node.execute('sudo tc qdisc change dev ' + database2_node.get_interface(network_name="bridge_c2_db2").get_os_interface() + ' root tbf rate 1536kbit burst 32kbit latency 50ms')
database2_node.execute('sudo tc qdisc change dev ' + database2_node.get_interface(network_name="bridge_c1_db2").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')

('', '')

In [6]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["27.973034155368804"], "Average_Service_Time_round_robin": ["15.01637064230442"], "Completion_Time_round_robin": [767.9930021762848], "Recent_Average_Service_Time_round_robin_per_function": ["25.91482949256897"], "Average_Service_Time_round_robin_per_function": ["13.921855271220208"], "Completion_Time_round_robin_per_function": [709.7204048633575], "Recent_Average_Service_Time_lbcf": ["16.362412214279175"], "Average_Service_Time_lbcf": ["14.423075640439988"], "Completion_Time_lbcf": [732.51242852211], "Recent_Average_Service_Time_profiling_only": ["16.891149961948393"], "Average_Service_Time_profiling_only": ["9.25455449640751"], "Completion_Time_profiling_only": [471.0954432487488], "Recent_Average_Service_Time_our_scheduler": ["8.666913211345673"], "Average_Service_Time_our_scheduler": ["10.309315202355386"], "Completion_Time_our_scheduler": [521.9396572113037], "other1

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["27.973034155368804"], "Average_Service_Time_round_robin": ["15.01637064230442"], "Completion_Time_round_robin": [767.9930021762848], "Recent_Average_Service_Time_round_robin_per_function": ["25.91482949256897"], "Average_Service_Time_round_robin_per_function": ["13.921855271220208"], "Completion_Time_round_robin_per_function": [709.7204048633575], "Recent_Average_Service_Time_lbcf": ["16.362412214279175"], "Average_Service_Time_lbcf": ["14.423075640439988"], "Completion_Time_lbcf": [732.51242852211], "Recent_Average_Service_Time_profiling_only": ["16.891149961948393"], "Average_Service_Time_profiling_only": ["9.25455449640751"], "Completion_Time_profiling_only": [471.0954432487488], "Recent_Average_Service_Time_our_scheduler": ["8.666913211345673"], "Average_Service_Time_our_scheduler": ["10.309315202355386"], "Completion_Time_our_scheduler": [521.9396572113037], 

In [15]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["28.00270805358887"], "Average_Service_Time_round_robin": ["15.09780116558075"], "Completion_Time_round_robin": [773.0155553817749], "Recent_Average_Service_Time_round_robin_per_function": ["25.946049773693083"], "Average_Service_Time_round_robin_per_function": ["13.98537291431427"], "Completion_Time_round_robin_per_function": [712.1864380836487], "Recent_Average_Service_Time_lbcf": ["13.968892049789428"], "Average_Service_Time_lbcf": ["14.343620447516441"], "Completion_Time_lbcf": [728.7940232753754], "Recent_Average_Service_Time_profiling_only": ["16.79369729757309"], "Average_Service_Time_profiling_only": ["9.232385621070861"], "Completion_Time_profiling_only": [469.94099140167236], "Recent_Average_Service_Time_our_scheduler": ["10.285843324661254"], "Average_Service_Time_our_scheduler": ["10.528461857914925"], "Completion_Time_our_scheduler": [532.8618562221527], "oth

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["28.00270805358887"], "Average_Service_Time_round_robin": ["15.09780116558075"], "Completion_Time_round_robin": [773.0155553817749], "Recent_Average_Service_Time_round_robin_per_function": ["25.946049773693083"], "Average_Service_Time_round_robin_per_function": ["13.98537291431427"], "Completion_Time_round_robin_per_function": [712.1864380836487], "Recent_Average_Service_Time_lbcf": ["13.968892049789428"], "Average_Service_Time_lbcf": ["14.343620447516441"], "Completion_Time_lbcf": [728.7940232753754], "Recent_Average_Service_Time_profiling_only": ["16.79369729757309"], "Average_Service_Time_profiling_only": ["9.232385621070861"], "Completion_Time_profiling_only": [469.94099140167236], "Recent_Average_Service_Time_our_scheduler": ["10.285843324661254"], "Average_Service_Time_our_scheduler": ["10.528461857914925"], "Completion_Time_our_scheduler": [532.8618562221527

In [16]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["27.197266709804534"], "Average_Service_Time_round_robin": ["14.869118968129158"], "Completion_Time_round_robin": [760.6846151351929], "Recent_Average_Service_Time_round_robin_per_function": ["26.511624383926392"], "Average_Service_Time_round_robin_per_function": ["13.982309992790222"], "Completion_Time_round_robin_per_function": [712.0304012298584], "Recent_Average_Service_Time_lbcf": ["15.767496037483216"], "Average_Service_Time_lbcf": ["14.35007855629921"], "Completion_Time_lbcf": [727.1601483821869], "Recent_Average_Service_Time_profiling_only": ["16.761890053749084"], "Average_Service_Time_profiling_only": ["9.29648736858368"], "Completion_Time_profiling_only": [473.1733901500702], "Recent_Average_Service_Time_our_scheduler": ["9.822853350639344"], "Average_Service_Time_our_scheduler": ["10.679978212356568"], "Completion_Time_our_scheduler": [540.5735557079315], "oth

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["27.197266709804534"], "Average_Service_Time_round_robin": ["14.869118968129158"], "Completion_Time_round_robin": [760.6846151351929], "Recent_Average_Service_Time_round_robin_per_function": ["26.511624383926392"], "Average_Service_Time_round_robin_per_function": ["13.982309992790222"], "Completion_Time_round_robin_per_function": [712.0304012298584], "Recent_Average_Service_Time_lbcf": ["15.767496037483216"], "Average_Service_Time_lbcf": ["14.35007855629921"], "Completion_Time_lbcf": [727.1601483821869], "Recent_Average_Service_Time_profiling_only": ["16.761890053749084"], "Average_Service_Time_profiling_only": ["9.29648736858368"], "Completion_Time_profiling_only": [473.1733901500702], "Recent_Average_Service_Time_our_scheduler": ["9.822853350639344"], "Average_Service_Time_our_scheduler": ["10.679978212356568"], "Completion_Time_our_scheduler": [540.5735557079315

In [17]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["27.930121517181398"], "Average_Service_Time_round_robin": ["15.026881850481033"], "Completion_Time_round_robin": [768.6660811901093], "Recent_Average_Service_Time_round_robin_per_function": ["26.076938652992247"], "Average_Service_Time_round_robin_per_function": ["13.988165687561034"], "Completion_Time_round_robin_per_function": [712.1117024421692], "Recent_Average_Service_Time_lbcf": ["16.36849318742752"], "Average_Service_Time_lbcf": ["14.365030411362648"], "Completion_Time_lbcf": [728.3119702339172], "Recent_Average_Service_Time_profiling_only": ["16.705103504657746"], "Average_Service_Time_profiling_only": ["9.24857190489769"], "Completion_Time_profiling_only": [470.76152992248535], "Recent_Average_Service_Time_our_scheduler": ["8.532740008831023"], "Average_Service_Time_our_scheduler": ["11.014714207172394"], "Completion_Time_our_scheduler": [555.2462050914764], "ot

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["27.930121517181398"], "Average_Service_Time_round_robin": ["15.026881850481033"], "Completion_Time_round_robin": [768.6660811901093], "Recent_Average_Service_Time_round_robin_per_function": ["26.076938652992247"], "Average_Service_Time_round_robin_per_function": ["13.988165687561034"], "Completion_Time_round_robin_per_function": [712.1117024421692], "Recent_Average_Service_Time_lbcf": ["16.36849318742752"], "Average_Service_Time_lbcf": ["14.365030411362648"], "Completion_Time_lbcf": [728.3119702339172], "Recent_Average_Service_Time_profiling_only": ["16.705103504657746"], "Average_Service_Time_profiling_only": ["9.24857190489769"], "Completion_Time_profiling_only": [470.76152992248535], "Recent_Average_Service_Time_our_scheduler": ["8.532740008831023"], "Average_Service_Time_our_scheduler": ["11.014714207172394"], "Completion_Time_our_scheduler": [555.246205091476

In [18]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["28.27577987909317"], "Average_Service_Time_round_robin": ["14.981933843612671"], "Completion_Time_round_robin": [766.5737974643707], "Recent_Average_Service_Time_round_robin_per_function": ["23.66197112798691"], "Average_Service_Time_round_robin_per_function": ["14.005409344792366"], "Completion_Time_round_robin_per_function": [713.0975787639618], "Recent_Average_Service_Time_lbcf": ["15.170386219024659"], "Average_Service_Time_lbcf": ["14.375476303696633"], "Completion_Time_lbcf": [729.9717328548431], "Recent_Average_Service_Time_profiling_only": ["16.776775681972502"], "Average_Service_Time_profiling_only": ["9.215565123438836"], "Completion_Time_profiling_only": [469.1162967681885], "Recent_Average_Service_Time_our_scheduler": ["9.647391259670258"], "Average_Service_Time_our_scheduler": ["11.243321195721625"], "Completion_Time_our_scheduler": [567.2214074134827], "oth

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["28.27577987909317"], "Average_Service_Time_round_robin": ["14.981933843612671"], "Completion_Time_round_robin": [766.5737974643707], "Recent_Average_Service_Time_round_robin_per_function": ["23.66197112798691"], "Average_Service_Time_round_robin_per_function": ["14.005409344792366"], "Completion_Time_round_robin_per_function": [713.0975787639618], "Recent_Average_Service_Time_lbcf": ["15.170386219024659"], "Average_Service_Time_lbcf": ["14.375476303696633"], "Completion_Time_lbcf": [729.9717328548431], "Recent_Average_Service_Time_profiling_only": ["16.776775681972502"], "Average_Service_Time_profiling_only": ["9.215565123438836"], "Completion_Time_profiling_only": [469.1162967681885], "Recent_Average_Service_Time_our_scheduler": ["9.647391259670258"], "Average_Service_Time_our_scheduler": ["11.243321195721625"], "Completion_Time_our_scheduler": [567.2214074134827

## x4 Performance Dispersion

In [19]:
cloud1_node.execute('sudo tc qdisc change dev ' + cloud1_node.get_interface(network_name="bridge_c1_db1").get_os_interface() + ' root tbf rate 2304kbit burst 32kbit latency 50ms')
cloud1_node.execute('sudo tc qdisc change dev ' + cloud1_node.get_interface(network_name="bridge_c1_db2").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
cloud2_node.execute('sudo tc qdisc change dev ' + cloud2_node.get_interface(network_name="bridge_c2_db2").get_os_interface() + ' root tbf rate 2304kbit burst 32kbit latency 50ms')
cloud2_node.execute('sudo tc qdisc change dev ' + cloud2_node.get_interface(network_name="bridge_c2_db1").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
database1_node.execute('sudo tc qdisc change dev ' + database1_node.get_interface(network_name="bridge_c1_db1").get_os_interface() + ' root tbf rate 2304kbit burst 32kbit latency 50ms')
database1_node.execute('sudo tc qdisc change dev ' + database1_node.get_interface(network_name="bridge_c2_db1").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')
database2_node.execute('sudo tc qdisc change dev ' + database2_node.get_interface(network_name="bridge_c2_db2").get_os_interface() + ' root tbf rate 2304kbit burst 32kbit latency 50ms')
database2_node.execute('sudo tc qdisc change dev ' + database2_node.get_interface(network_name="bridge_c1_db2").get_os_interface() + ' root tbf rate 512kbit burst 32kbit latency 50ms')

('', '')

In [8]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["27.787100994586943"], "Average_Service_Time_round_robin": ["14.830071491718293"], "Completion_Time_round_robin": [758.6953637599945], "Recent_Average_Service_Time_round_robin_per_function": ["26.112835741043092"], "Average_Service_Time_round_robin_per_function": ["13.984463100075722"], "Completion_Time_round_robin_per_function": [712.2604525089264], "Recent_Average_Service_Time_lbcf": ["13.134730947017669"], "Average_Service_Time_lbcf": ["13.55786049091816"], "Completion_Time_lbcf": [689.1710398197174], "Recent_Average_Service_Time_profiling_only": ["11.102313721179963"], "Average_Service_Time_profiling_only": ["6.220269649386406"], "Completion_Time_profiling_only": [316.55676436424255], "Recent_Average_Service_Time_our_scheduler": ["6.579293119907379"], "Average_Service_Time_our_scheduler": ["9.05251325070858"], "Completion_Time_our_scheduler": [456.7807686328888], "oth

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["27.787100994586943"], "Average_Service_Time_round_robin": ["14.830071491718293"], "Completion_Time_round_robin": [758.6953637599945], "Recent_Average_Service_Time_round_robin_per_function": ["26.112835741043092"], "Average_Service_Time_round_robin_per_function": ["13.984463100075722"], "Completion_Time_round_robin_per_function": [712.2604525089264], "Recent_Average_Service_Time_lbcf": ["13.134730947017669"], "Average_Service_Time_lbcf": ["13.55786049091816"], "Completion_Time_lbcf": [689.1710398197174], "Recent_Average_Service_Time_profiling_only": ["11.102313721179963"], "Average_Service_Time_profiling_only": ["6.220269649386406"], "Completion_Time_profiling_only": [316.55676436424255], "Recent_Average_Service_Time_our_scheduler": ["6.579293119907379"], "Average_Service_Time_our_scheduler": ["9.05251325070858"], "Completion_Time_our_scheduler": [456.7807686328888

In [20]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["27.558788764476777"], "Average_Service_Time_round_robin": ["14.822981609225273"], "Completion_Time_round_robin": [758.253534078598], "Recent_Average_Service_Time_round_robin_per_function": ["26.041210734844206"], "Average_Service_Time_round_robin_per_function": ["14.098147036910056"], "Completion_Time_round_robin_per_function": [718.4614655971527], "Recent_Average_Service_Time_lbcf": ["13.578636920452118"], "Average_Service_Time_lbcf": ["13.710574469923973"], "Completion_Time_lbcf": [695.653781414032], "Recent_Average_Service_Time_profiling_only": ["11.226382768154144"], "Average_Service_Time_profiling_only": ["6.170395400524139"], "Completion_Time_profiling_only": [314.0652639865875], "Recent_Average_Service_Time_our_scheduler": ["7.853925788402558"], "Average_Service_Time_our_scheduler": ["9.109051489949227"], "Completion_Time_our_scheduler": [460.2008078098297], "othe

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["27.558788764476777"], "Average_Service_Time_round_robin": ["14.822981609225273"], "Completion_Time_round_robin": [758.253534078598], "Recent_Average_Service_Time_round_robin_per_function": ["26.041210734844206"], "Average_Service_Time_round_robin_per_function": ["14.098147036910056"], "Completion_Time_round_robin_per_function": [718.4614655971527], "Recent_Average_Service_Time_lbcf": ["13.578636920452118"], "Average_Service_Time_lbcf": ["13.710574469923973"], "Completion_Time_lbcf": [695.653781414032], "Recent_Average_Service_Time_profiling_only": ["11.226382768154144"], "Average_Service_Time_profiling_only": ["6.170395400524139"], "Completion_Time_profiling_only": [314.0652639865875], "Recent_Average_Service_Time_our_scheduler": ["7.853925788402558"], "Average_Service_Time_our_scheduler": ["9.109051489949227"], "Completion_Time_our_scheduler": [460.2008078098297]

In [21]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["27.659446835517883"], "Average_Service_Time_round_robin": ["14.72819494664669"], "Completion_Time_round_robin": [753.6576459407806], "Recent_Average_Service_Time_round_robin_per_function": ["25.229020869731904"], "Average_Service_Time_round_robin_per_function": ["14.049154955267905"], "Completion_Time_round_robin_per_function": [715.3918962478638], "Recent_Average_Service_Time_lbcf": ["20.330597376823427"], "Average_Service_Time_lbcf": ["14.215542348742485"], "Completion_Time_lbcf": [741.481698513031], "Recent_Average_Service_Time_profiling_only": ["11.1752037525177"], "Average_Service_Time_profiling_only": ["6.232890077352524"], "Completion_Time_profiling_only": [317.1801280975342], "Recent_Average_Service_Time_our_scheduler": ["5.731502592563629"], "Average_Service_Time_our_scheduler": ["9.531140170454979"], "Completion_Time_our_scheduler": [479.5945484638214], "other1

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["27.659446835517883"], "Average_Service_Time_round_robin": ["14.72819494664669"], "Completion_Time_round_robin": [753.6576459407806], "Recent_Average_Service_Time_round_robin_per_function": ["25.229020869731904"], "Average_Service_Time_round_robin_per_function": ["14.049154955267905"], "Completion_Time_round_robin_per_function": [715.3918962478638], "Recent_Average_Service_Time_lbcf": ["20.330597376823427"], "Average_Service_Time_lbcf": ["14.215542348742485"], "Completion_Time_lbcf": [741.481698513031], "Recent_Average_Service_Time_profiling_only": ["11.1752037525177"], "Average_Service_Time_profiling_only": ["6.232890077352524"], "Completion_Time_profiling_only": [317.1801280975342], "Recent_Average_Service_Time_our_scheduler": ["5.731502592563629"], "Average_Service_Time_our_scheduler": ["9.531140170454979"], "Completion_Time_our_scheduler": [479.5945484638214], 

In [22]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["28.0793568611145"], "Average_Service_Time_round_robin": ["14.670811810016632"], "Completion_Time_round_robin": [750.7348005771637], "Recent_Average_Service_Time_round_robin_per_function": ["25.103172981739043"], "Average_Service_Time_round_robin_per_function": ["13.960096792459488"], "Completion_Time_round_robin_per_function": [711.0627529621124], "Recent_Average_Service_Time_lbcf": ["14.911529231071473"], "Average_Service_Time_lbcf": ["13.929673098683358"], "Completion_Time_lbcf": [706.3571608066559], "Recent_Average_Service_Time_profiling_only": ["11.102631688117981"], "Average_Service_Time_profiling_only": ["6.206873598694801"], "Completion_Time_profiling_only": [315.8915991783142], "Recent_Average_Service_Time_our_scheduler": ["6.0698969006538395"], "Average_Service_Time_our_scheduler": ["9.16360174357891"], "Completion_Time_our_scheduler": [460.90230226516724], "oth

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["28.0793568611145"], "Average_Service_Time_round_robin": ["14.670811810016632"], "Completion_Time_round_robin": [750.7348005771637], "Recent_Average_Service_Time_round_robin_per_function": ["25.103172981739043"], "Average_Service_Time_round_robin_per_function": ["13.960096792459488"], "Completion_Time_round_robin_per_function": [711.0627529621124], "Recent_Average_Service_Time_lbcf": ["14.911529231071473"], "Average_Service_Time_lbcf": ["13.929673098683358"], "Completion_Time_lbcf": [706.3571608066559], "Recent_Average_Service_Time_profiling_only": ["11.102631688117981"], "Average_Service_Time_profiling_only": ["6.206873598694801"], "Completion_Time_profiling_only": [315.8915991783142], "Recent_Average_Service_Time_our_scheduler": ["6.0698969006538395"], "Average_Service_Time_our_scheduler": ["9.16360174357891"], "Completion_Time_our_scheduler": [460.90230226516724

In [23]:
scheduler_node.execute('python3 -u main.py all |& tee output_scheduler.txt')

round_robin
round_robin_per_function
lbcf
profiling_only
our_scheduler
{"Recent_Average_Service_Time_round_robin": ["28.905455327033998"], "Average_Service_Time_round_robin": ["14.9196435393095"], "Completion_Time_round_robin": [763.8618297576904], "Recent_Average_Service_Time_round_robin_per_function": ["25.847651743888854"], "Average_Service_Time_round_robin_per_function": ["14.118361803650856"], "Completion_Time_round_robin_per_function": [719.5419449806213], "Recent_Average_Service_Time_lbcf": ["13.709705877304078"], "Average_Service_Time_lbcf": ["2.950728263378143"], "Completion_Time_lbcf": [158.1785318851471], "Recent_Average_Service_Time_profiling_only": ["0.9163980722427368"], "Average_Service_Time_profiling_only": ["0.9221286851167679"], "Completion_Time_profiling_only": [46.54212212562561], "Recent_Average_Service_Time_our_scheduler": ["0.861646568775177"], "Average_Service_Time_our_scheduler": ["0.8891865824460983"], "Completion_Time_our_scheduler": [44.8907687664032], "othe

('round_robin\nround_robin_per_function\nlbcf\nprofiling_only\nour_scheduler\n{"Recent_Average_Service_Time_round_robin": ["28.905455327033998"], "Average_Service_Time_round_robin": ["14.9196435393095"], "Completion_Time_round_robin": [763.8618297576904], "Recent_Average_Service_Time_round_robin_per_function": ["25.847651743888854"], "Average_Service_Time_round_robin_per_function": ["14.118361803650856"], "Completion_Time_round_robin_per_function": [719.5419449806213], "Recent_Average_Service_Time_lbcf": ["13.709705877304078"], "Average_Service_Time_lbcf": ["2.950728263378143"], "Completion_Time_lbcf": [158.1785318851471], "Recent_Average_Service_Time_profiling_only": ["0.9163980722427368"], "Average_Service_Time_profiling_only": ["0.9221286851167679"], "Completion_Time_profiling_only": [46.54212212562561], "Recent_Average_Service_Time_our_scheduler": ["0.861646568775177"], "Average_Service_Time_our_scheduler": ["0.8891865824460983"], "Completion_Time_our_scheduler": [44.8907687664032]

### x2 Dispersion

---

{"Recent_Average_Service_Time_round_robin": ["27.685994970798493"],

"Average_Service_Time_round_robin": ["15.145879358172417"],

"Completion_Time_round_robin": [774.4025707244873],

---

"Recent_Average_Service_Time_round_robin_per_function": ["25.497785210609436"],

"Average_Service_Time_round_robin_per_function": ["13.99199562251568"],

"Completion_Time_round_robin_per_function": [712.3744764328003],

---

"Recent_Average_Service_Time_lbcf": ["16.241431510448457"],

"Average_Service_Time_lbcf": ["15.326708951830865"],

"Completion_Time_lbcf": [778.6501080989838],

---

"Recent_Average_Service_Time_profiling_only": ["20.527017688751222"],

"Average_Service_Time_profiling_only": ["14.11012697517872"],

"Completion_Time_profiling_only": [715.6847639083862],

---

"Recent_Average_Service_Time_our_scheduler": ["11.465923464298248"],

"Average_Service_Time_our_scheduler": ["13.830918514847756"],

"Completion_Time_our_scheduler": [698.0407493114471],

---

"other1": "start wall clock time = 1683752724.2464843
current time = 1683753498.6490495
start wall clock time = 1683753498.662724
current time = 1683754211.0371945
start wall clock time = 1683754211.0471952
current time = 1683754989.6972976
start wall clock time = 1683754989.7093318
current time = 1683755705.39409
start wall clock time = 1683755705.4071875
current time = 1683756403.4479308
", "other2": "", "other3": ""}


---

{"Recent_Average_Service_Time_round_robin": ["27.81156152486801"],

"Average_Service_Time_round_robin": ["15.513222743988036"],

"Completion_Time_round_robin": [792.9535021781921],

---

"Recent_Average_Service_Time_round_robin_per_function": ["25.381979632377625"], 

"Average_Service_Time_round_robin_per_function": ["13.980377514958382"], 

"Completion_Time_round_robin_per_function": [711.8785898685455],

---

"Recent_Average_Service_Time_lbcf": ["16.980079853534697"],

"Average_Service_Time_lbcf": ["14.81700963127613"],

"Completion_Time_lbcf": [750.5438272953033],

---

"Recent_Average_Service_Time_profiling_only": ["26.038368141651155"],

"Average_Service_Time_profiling_only": ["14.386791252613067"],

"Completion_Time_profiling_only": [730.9428195953369],

---

"Recent_Average_Service_Time_our_scheduler": ["12.389434146881104"],

"Average_Service_Time_our_scheduler": ["13.237221311092377"],

"Completion_Time_our_scheduler": [668.3888969421387],

---

"other1": "start wall clock time = 1683768974.3236034
current time = 1683769767.2770958
start wall clock time = 1683769767.2892752
current time = 1683770479.1678588
start wall clock time = 1683770479.180913
current time = 1683771229.7247329
start wall clock time = 1683771229.7356305
current time = 1683771960.6784434
start wall clock time = 1683771960.6898546
current time = 1683772629.0787458
", "other2": "", "other3": ""}

{"Recent_Average_Service_Time_round_robin": ["26.05106452703476"],

"Average_Service_Time_round_robin": ["15.103995965242387"],

"Completion_Time_round_robin": [772.2252774238586],

---

"Recent_Average_Service_Time_round_robin_per_function": ["25.263446485996248"], 

"Average_Service_Time_round_robin_per_function": ["14.29089435350895"], 

"Completion_Time_round_robin_per_function": [727.3632688522339],

---

"Recent_Average_Service_Time_lbcf": ["15.3405207157135"],

"Average_Service_Time_lbcf": ["15.115064324378967"],

"Completion_Time_lbcf": [768.1816074848175],

---

"Recent_Average_Service_Time_profiling_only": ["18.831792688369752"],

"Average_Service_Time_profiling_only": ["14.441861746549606"],

"Completion_Time_profiling_only": [731.3837289810181],

---

"Recent_Average_Service_Time_our_scheduler": ["11.312145292758942"],

"Average_Service_Time_our_scheduler": ["12.44009664785862"],

"Completion_Time_our_scheduler": [628.8954129219055],

---

"other1": "start wall clock time = 1683772630.9160328
current time = 1683773403.141301
start wall clock time = 1683773403.1551023
current time = 1683774130.5183659
start wall clock time = 1683774130.5292902
current time = 1683774898.7108877
start wall clock time = 1683774898.7236912
current time = 1683775630.1074104
start wall clock time = 1683775630.116858
current time = 1683776259.0122647
", "other2": "", "other3": ""}

---

{"Recent_Average_Service_Time_round_robin": ["27.234154677391054"],

"Average_Service_Time_round_robin": ["15.410624063014984"],

"Completion_Time_round_robin": [787.6461882591248],

---

"Recent_Average_Service_Time_round_robin_per_function": ["25.060973525047302"],

"Average_Service_Time_round_robin_per_function": ["13.976637208580971"], 

"Completion_Time_round_robin_per_function": [712.7597382068634],

---

"Recent_Average_Service_Time_lbcf": ["15.122440314292907"],

"Average_Service_Time_lbcf": ["14.7123236951828"],

"Completion_Time_lbcf": [742.606772184372],

---

"Recent_Average_Service_Time_profiling_only": ["25.246407687664032"],

"Average_Service_Time_profiling_only": ["15.866463209748268"],

"Completion_Time_profiling_only": [805.7894582748413],

---

"Recent_Average_Service_Time_our_scheduler": ["11.87836035490036"],

"Average_Service_Time_our_scheduler": ["12.65770934844017"],

"Completion_Time_our_scheduler": [639.3929443359375],

---

"other1": "start wall clock time = 1683776260.8779626
current time = 1683777048.5241451
start wall clock time = 1683777048.5355773
current time = 1683777761.29531
start wall clock time = 1683777761.3082478
current time = 1683778503.9150145
start wall clock time = 1683778503.9270873
current time = 1683779309.7165391
start wall clock time = 1683779309.728501
current time = 1683779949.1214366
", "other2": "", "other3": ""}

---

{"Recent_Average_Service_Time_round_robin": ["30.43135303258896"],

"Average_Service_Time_round_robin": ["15.347558294177055"],

"Completion_Time_round_robin": [785.2355532646179],

---

"Recent_Average_Service_Time_round_robin_per_function": ["26.316376745700836"], 

"Average_Service_Time_round_robin_per_function": ["14.269409460425377"], 

"Completion_Time_round_robin_per_function": [726.3734467029572],

---

"Recent_Average_Service_Time_lbcf": ["15.144583404064178"],

"Average_Service_Time_lbcf": ["15.191838337421418"],

"Completion_Time_lbcf": [769.1439583301544],

---

"Recent_Average_Service_Time_profiling_only": ["22.74024988412857"],

"Average_Service_Time_profiling_only": ["14.255639310002326"],

"Completion_Time_profiling_only": [723.411340713501],

---

"Recent_Average_Service_Time_our_scheduler": ["12.555805289745331"],

"Average_Service_Time_our_scheduler": ["12.364351061940193"],

"Completion_Time_our_scheduler": [625.5389659404755],

---

"other1": "start wall clock time = 1683779950.8829522
current time = 1683780736.1184993
start wall clock time = 1683780736.127317
current time = 1683781462.5007577
start wall clock time = 1683781462.5131135
current time = 1683782231.6570659
start wall clock time = 1683782231.6656637
current time = 1683782955.0769987
start wall clock time = 1683782955.0872095
current time = 1683783580.6261702
", "other2": "", "other3": ""}

### x3 Dispersion

{"Recent_Average_Service_Time_round_robin": ["27.973034155368804"],

"Average_Service_Time_round_robin": ["15.01637064230442"],

"Completion_Time_round_robin": [767.9930021762848],

---

"Recent_Average_Service_Time_round_robin_per_function": ["25.91482949256897"],

"Average_Service_Time_round_robin_per_function": ["13.921855271220208"],

"Completion_Time_round_robin_per_function": [709.7204048633575],

---

"Recent_Average_Service_Time_lbcf": ["16.362412214279175"],

"Average_Service_Time_lbcf": ["14.423075640439988"],

"Completion_Time_lbcf": [732.51242852211],

---

"Recent_Average_Service_Time_profiling_only": ["16.891149961948393"],

"Average_Service_Time_profiling_only": ["9.25455449640751"],

"Completion_Time_profiling_only": [471.0954432487488],

---

"Recent_Average_Service_Time_our_scheduler": ["8.666913211345673"],

"Average_Service_Time_our_scheduler": ["10.309315202355386"],

"Completion_Time_our_scheduler": [521.9396572113037],

---

"other1": "start wall clock time = 1683757624.2481594\\ncurrent time = 1683758392.2411547\\nstart wall clock time = 1683758392.2536502\\ncurrent time = 1683759101.974049\\nstart wall clock time = 1683759101.9885166\\ncurrent time = 1683759834.50094\\nstart wall clock time = 1683759834.511362\\ncurrent time = 1683760305.606799\\nstart wall clock time = 1683760305.618058\\ncurrent time = 1683760827.557703\\n", "other2": "", "other3": ""}

---

{"Recent_Average_Service_Time_round_robin": ["28.00270805358887"],

"Average_Service_Time_round_robin": ["15.09780116558075"],

"Completion_Time_round_robin": [773.0155553817749],

---

"Recent_Average_Service_Time_round_robin_per_function": ["25.946049773693083"],

"Average_Service_Time_round_robin_per_function": ["13.98537291431427"],

"Completion_Time_round_robin_per_function": [712.1864380836487],

---

"Recent_Average_Service_Time_lbcf": ["13.968892049789428"],

"Average_Service_Time_lbcf": ["14.343620447516441"],

"Completion_Time_lbcf": [728.7940232753754],

---

"Recent_Average_Service_Time_profiling_only": ["16.79369729757309"],

"Average_Service_Time_profiling_only": ["9.232385621070861"],

"Completion_Time_profiling_only": [469.94099140167236],

---

"Recent_Average_Service_Time_our_scheduler": ["10.285843324661254"],

"Average_Service_Time_our_scheduler": ["10.528461857914925"],

"Completion_Time_our_scheduler": [532.8618562221527],

---

"other1": "start wall clock time = 1683783594.6848607
current time = 1683784367.7004101
start wall clock time = 1683784367.7122912
current time = 1683785079.898723
start wall clock time = 1683785079.9115174
current time = 1683785808.7055304
start wall clock time = 1683785808.716728
current time = 1683786278.657712
start wall clock time = 1683786278.669183
current time = 1683786811.5310328
", "other2": "", "other3": ""}

---

{"Recent_Average_Service_Time_round_robin": ["27.197266709804534"],

"Average_Service_Time_round_robin": ["14.869118968129158"],

"Completion_Time_round_robin": [760.6846151351929],

---

"Recent_Average_Service_Time_round_robin_per_function": ["26.511624383926392"],

"Average_Service_Time_round_robin_per_function": ["13.982309992790222"],

"Completion_Time_round_robin_per_function": [712.0304012298584],

---

"Recent_Average_Service_Time_lbcf": ["15.767496037483216"],

"Average_Service_Time_lbcf": ["14.35007855629921"],

"Completion_Time_lbcf": [727.1601483821869],

---

"Recent_Average_Service_Time_profiling_only": ["16.761890053749084"],

"Average_Service_Time_profiling_only": ["9.29648736858368"],

"Completion_Time_profiling_only": [473.1733901500702],

---

"Recent_Average_Service_Time_our_scheduler": ["9.822853350639344"],

"Average_Service_Time_our_scheduler": ["10.679978212356568"],

"Completion_Time_our_scheduler": [540.5735557079315],

---

"other1": "start wall clock time = 1683786813.0500534
current time = 1683787573.7346623
start wall clock time = 1683787573.7462947
current time = 1683788285.776691
start wall clock time = 1683788285.78814
current time = 1683789012.9482794
start wall clock time = 1683789012.9638793
current time = 1683789486.1372619
start wall clock time = 1683789486.1537642
current time = 1683790026.72731
", "other2": "", "other3": ""}

---

{"Recent_Average_Service_Time_round_robin": ["27.930121517181398"],

"Average_Service_Time_round_robin": ["15.026881850481033"],

"Completion_Time_round_robin": [768.6660811901093],

---

"Recent_Average_Service_Time_round_robin_per_function": ["26.076938652992247"],

"Average_Service_Time_round_robin_per_function": ["13.988165687561034"],

"Completion_Time_round_robin_per_function": [712.1117024421692],

---

"Recent_Average_Service_Time_lbcf": ["16.36849318742752"],

"Average_Service_Time_lbcf": ["14.365030411362648"],

"Completion_Time_lbcf": [728.3119702339172],

---

"Recent_Average_Service_Time_profiling_only": ["16.705103504657746"],

"Average_Service_Time_profiling_only": ["9.24857190489769"],

"Completion_Time_profiling_only": [470.76152992248535],

---

"Recent_Average_Service_Time_our_scheduler": ["8.532740008831023"],

"Average_Service_Time_our_scheduler": ["11.014714207172394"],

"Completion_Time_our_scheduler": [555.2462050914764],

---

"other1": "start wall clock time = 1683790028.249186
current time = 1683790796.9152575
start wall clock time = 1683790796.9268863
current time = 1683791509.038583
start wall clock time = 1683791509.0502243
current time = 1683792237.3621888
start wall clock time = 1683792237.3705356
current time = 1683792708.1320558
start wall clock time = 1683792708.1422443
current time = 1683793263.3884392
", "other2": "", "other3": ""}

---

{"Recent_Average_Service_Time_round_robin": ["28.27577987909317"],

"Average_Service_Time_round_robin": ["14.981933843612671"],

"Completion_Time_round_robin": [766.5737974643707],

---

"Recent_Average_Service_Time_round_robin_per_function": ["23.66197112798691"],

"Average_Service_Time_round_robin_per_function": ["14.005409344792366"],

"Completion_Time_round_robin_per_function": [713.0975787639618],

---

"Recent_Average_Service_Time_lbcf": ["15.170386219024659"],

"Average_Service_Time_lbcf": ["14.375476303696633"],

"Completion_Time_lbcf": [729.9717328548431],

---

"Recent_Average_Service_Time_profiling_only": ["16.776775681972502"],

"Average_Service_Time_profiling_only": ["9.215565123438836"],

"Completion_Time_profiling_only": [469.1162967681885],

---

"Recent_Average_Service_Time_our_scheduler": ["9.647391259670258"],

"Average_Service_Time_our_scheduler": ["11.243321195721625"],

"Completion_Time_our_scheduler": [567.2214074134827],

---

"other1": "start wall clock time = 1683793264.895683
current time = 1683794031.4694734
start wall clock time = 1683794031.4885106
current time = 1683794744.5860841
start wall clock time = 1683794744.597925
current time = 1683795474.5696523
start wall clock time = 1683795474.5824804
current time = 1683795943.6987677
start wall clock time = 1683795943.7097893
current time = 1683796510.9311903
", "other2": "", "other3": ""}

### x4 Dispersion

{"Recent_Average_Service_Time_round_robin": ["27.787100994586943"],

"Average_Service_Time_round_robin": ["14.830071491718293"],

"Completion_Time_round_robin": [758.6953637599945],

---

"Recent_Average_Service_Time_round_robin_per_function": ["26.112835741043092"],

"Average_Service_Time_round_robin_per_function": ["13.984463100075722"],

"Completion_Time_round_robin_per_function": [712.2604525089264],

---

"Recent_Average_Service_Time_lbcf": ["13.134730947017669"],

"Average_Service_Time_lbcf": ["13.55786049091816"],

"Completion_Time_lbcf": [689.1710398197174],

---

"Recent_Average_Service_Time_profiling_only": ["11.102313721179963"],

"Average_Service_Time_profiling_only": ["6.220269649386406"],

"Completion_Time_profiling_only": [316.55676436424255],

---

"Recent_Average_Service_Time_our_scheduler": ["6.579293119907379"],

"Average_Service_Time_our_scheduler": ["9.05251325070858"],

"Completion_Time_our_scheduler": [456.7807686328888],

---

"other1": "start wall clock time = 1683760841.1853576
current time = 1683761599.8807163
start wall clock time = 1683761599.891437
current time = 1683762312.1518838
start wall clock time = 1683762312.163584
current time = 1683763001.3346148
start wall clock time = 1683763001.343125
current time = 1683763317.8998795
start wall clock time = 1683763317.914097
current time = 1683763774.6948593
", "other2": "", "other3": ""}

---

{"Recent_Average_Service_Time_round_robin": ["27.558788764476777"],

"Average_Service_Time_round_robin": ["14.822981609225273"],

"Completion_Time_round_robin": [758.253534078598],

---

"Recent_Average_Service_Time_round_robin_per_function": ["26.041210734844206"],

"Average_Service_Time_round_robin_per_function": ["14.098147036910056"],

"Completion_Time_round_robin_per_function": [718.4614655971527],

---

"Recent_Average_Service_Time_lbcf": ["13.578636920452118"],

"Average_Service_Time_lbcf": ["13.710574469923973"],

"Completion_Time_lbcf": [695.653781414032],

---

"Recent_Average_Service_Time_profiling_only": ["11.226382768154144"],

"Average_Service_Time_profiling_only": ["6.170395400524139"],

"Completion_Time_profiling_only": [314.0652639865875],

---

"Recent_Average_Service_Time_our_scheduler": ["7.853925788402558"],

"Average_Service_Time_our_scheduler": ["9.109051489949227"],

"Completion_Time_our_scheduler": [460.2008078098297],

---

"other1": "start wall clock time = 1683796525.036275
current time = 1683797283.289799
start wall clock time = 1683797283.301208
current time = 1683798001.7626638
start wall clock time = 1683798001.7756178
current time = 1683798697.4293916
start wall clock time = 1683798697.4459505
current time = 1683799011.5112038
start wall clock time = 1683799011.5240262
current time = 1683799471.7248287
", "other2": "", "other3": ""}

---

{"Recent_Average_Service_Time_round_robin": ["27.659446835517883"],

"Average_Service_Time_round_robin": ["14.72819494664669"],

"Completion_Time_round_robin": [753.6576459407806],

---

"Recent_Average_Service_Time_round_robin_per_function": ["25.229020869731904"],

"Average_Service_Time_round_robin_per_function": ["14.049154955267905"],

"Completion_Time_round_robin_per_function": [715.3918962478638],

---

"Recent_Average_Service_Time_lbcf": ["20.330597376823427"],

"Average_Service_Time_lbcf": ["14.215542348742485"],

"Completion_Time_lbcf": [741.481698513031],

---

"Recent_Average_Service_Time_profiling_only": ["11.1752037525177"],

"Average_Service_Time_profiling_only": ["6.232890077352524"],

"Completion_Time_profiling_only": [317.1801280975342],

---

"Recent_Average_Service_Time_our_scheduler": ["5.731502592563629"],

"Average_Service_Time_our_scheduler": ["9.531140170454979"],

"Completion_Time_our_scheduler": [479.5945484638214],

---

"other1": "start wall clock time = 1683799473.2022796
current time = 1683800226.8599207
start wall clock time = 1683800226.8688297
current time = 1683800942.2607205
start wall clock time = 1683800942.2731767
current time = 1683801683.7548676
start wall clock time = 1683801683.7661548
current time = 1683802000.9462733
start wall clock time = 1683802000.9570286
current time = 1683802480.551571
", "other2": "", "other3": ""}

---

{"Recent_Average_Service_Time_round_robin": ["28.0793568611145"],

"Average_Service_Time_round_robin": ["14.670811810016632"],

"Completion_Time_round_robin": [750.7348005771637],

---

"Recent_Average_Service_Time_round_robin_per_function": ["25.103172981739043"],

"Average_Service_Time_round_robin_per_function": ["13.960096792459488"], 

"Completion_Time_round_robin_per_function": [711.0627529621124],

---

"Recent_Average_Service_Time_lbcf": ["14.911529231071473"],

"Average_Service_Time_lbcf": ["13.929673098683358"],

"Completion_Time_lbcf": [706.3571608066559],

---

"Recent_Average_Service_Time_profiling_only": ["11.102631688117981"],

"Average_Service_Time_profiling_only": ["6.206873598694801"],

"Completion_Time_profiling_only": [315.8915991783142],

---

"Recent_Average_Service_Time_our_scheduler": ["6.0698969006538395"],

"Average_Service_Time_our_scheduler": ["9.16360174357891"],

"Completion_Time_our_scheduler": [460.90230226516724],

---

"other1": "start wall clock time = 1683802482.3994696
current time = 1683803233.1342642
start wall clock time = 1683803233.1483078
current time = 1683803944.2110543
start wall clock time = 1683803944.2207563
current time = 1683804650.5779102
start wall clock time = 1683804650.5908022
current time = 1683804966.4823952
start wall clock time = 1683804966.4917448
current time = 1683805427.394041
", "other2": "", "other3": ""}